In [ ]:
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO

# function to read the model path (important for .exe builds)
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)


model_path = resource_path("best.pt")
model = YOLO(model_path)


root = tk.Tk()
root.title("Smart Behavior and Weapon Detection")
root.geometry("500x600")

Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

canvas = Canvas(root, width=400, height=400, bg="gray")
canvas.pack(pady=10)


file_label = Label(root, text="No file selected", fg="red")
file_label.pack()


result_label = Label(root, text="", fg="green")
result_label.pack()


cap = None
live_running = False


def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path:
        file_label.config(text=f"Selected: {file_path}")
        process_file(file_path)

def process_file(file_path):
    # Display image if it's a static file
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path)
        img.thumbnail((400, 400))
        img = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=img)
        canvas.image = img

    # Run detectionB
    results = model(file_path)
    results[0].show()  # for debugging
    results.save(save_dir="detections/")
    result_label.config(text="Detection Complete! Check 'detections/' folder.")

def start_live_detection():
    global cap, live_running
    cap = cv2.VideoCapture(0)
    live_running = True
    show_live_frame()

def stop_live_detection():
    global cap, live_running
    live_running = False
    if cap:
        cap.release()
    canvas.delete("all")
    result_label.config(text="Live detection stopped.")
    file_label.config(text="No file selected")

def show_live_frame():
    global cap, live_running
    if cap and live_running:
        ret, frame = cap.read()
        if not ret:
            stop_live_detection()
            return

        results = model(frame)[0]

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Convert frame to ImageTk
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

        root.after(10, show_live_frame)

# --- Buttons ---
Button(root, text="Select Image/Video", command=select_file).pack(pady=5)
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# --- Start the GUI loop ---
root.mainloop()


Function to start and stop live detection

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO

# Load YOLO model
model = YOLO("best.pt")

# Tkinter GUI setup
root = tk.Tk()
root.title("Live Weapon Detection")

# Create a Label to display frames
video_label = tk.Label(root)
video_label.pack()

cap = None  # Global video capture

def start_detection():
    global cap
    cap = cv2.VideoCapture(0)
    show_frame()

def show_frame():
    global cap

    ret, frame = cap.read()
    if not ret:
        return

    results = model(frame)[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame)
    imgtk = ImageTk.PhotoImage(image=img)

    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    video_label.after(10, show_frame)  # Schedule next frame

def stop_detection():
    global cap
    if cap:
        cap.release()
        video_label.config(image='')

# Buttons
start_btn = tk.Button(root, text="Start Live Detection", command=start_detection)
start_btn.pack(pady=5)

stop_btn = tk.Button(root, text="Stop", command=stop_detection)
stop_btn.pack(pady=5)

root.mainloop()


Everything combined for testing

In [ ]:
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO


# function to read the model to avoid errors in .exe files later
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best.pt")
model = YOLO(model_path)

# --- GUI setup ---
root = tk.Tk()
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("500x600") # size of the application window

# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path:
        file_label.config(text=f"Selected: {file_path}")
        process_file(file_path)

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None
live_running = False

# function to start live deteciton
def start_live_detection():
    global cap, live_running
    cap = cv2.VideoCapture(0)
    live_running = True
    show_live_frame()

# function to stop detection
def stop_live_detection():
    global cap, live_running
    live_running = False
    if cap:
        cap.release()
    canvas.delete("all")
    result_label.config(text="Live detection stopped.")
    file_label.config(text="No file selected")

# display frames of camera and run model upon
def show_live_frame():
    global cap, live_running
    if cap and live_running:
        ret, frame = cap.read()
        if not ret:
            stop_live_detection()
            return

        results = model(frame)[0]

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it

        root.after(10, show_live_frame)

# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# exit button
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# file label and canvas
file_label = Label(root, text="No file selected", fg="red") # path or name of the file selected by the user, if none is selected it displays text specified
file_label.pack()

canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # Display image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        img = ImageTk.PhotoImage(img) # converting image so tkinter can display
        canvas.create_image(200, 200, anchor=tk.CENTER, image=img) # display image on canvas
        canvas.image = img # Keeps the image saved so garbage collection dont remove it

    
    results = model(file_path, save=True, project="detections", name="results", exist_ok=True) # running model on the image and saving detection
    results[0].show() # display result in a seperate window
    # results.save(save_dir="detections/")
    # saved_path = results[0].save_dir
    # print("Saved at:", saved_path)

    result_label.config(text="Detection Complete! Check 'detections/' folder.")

# Start GUI loop
root.mainloop()

Use of threading

In [ ]:
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading


# function to read the model to avoid errors in .exe files later
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best.pt")
model = YOLO(model_path)

# --- GUI setup ---
root = tk.Tk()
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("500x600") # size of the application window

# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path:
        file_label.config(text=f"Selected: {file_path}")
        process_file(file_path)

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None
live_running = False

# function to start live deteciton
def start_live_detection():
    global cap, live_running
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        result_label.config(text="Camera not accessible")
        return
    live_running = True
    threading.Thread(target=show_live_frame, daemon=True).start()

# function to stop detection
def stop_live_detection():
    global cap, live_running
    live_running = False
    if cap:
        cap.release()
    canvas.delete("all")
    result_label.config(text="Live detection stopped.")
    file_label.config(text="No file selected")

# display frames of camera and run model upon
def show_live_frame():
    global cap, live_running
    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            stop_live_detection()
            break

        results = model(frame)[0]

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(400, 400, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        root.update_idletasks()
        root.update()

# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck
def clear_canvas():
    canvas.delete("all")
    canvas.image = None
    file_label.config(text="No file selected")
    result_label.config(text="")
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5)

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# file label and canvas
file_label = Label(root, text="No file selected", fg="red") # path or name of the file selected by the user, if none is selected it displays text specified
file_label.pack()

canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # Display image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        img = ImageTk.PhotoImage(img) # converting image so tkinter can display
        canvas.create_image(200, 200, anchor=tk.CENTER, image=img) # display image on canvas
        canvas.image = img # Keeps the image saved so garbage collection dont remove it

    results = model(file_path, save=True, project="detections", name="results", exist_ok=True) # running model on the image and saving detection
    results[0].show() # display result in a seperate window
    # results.save(save_dir="detections/")
    # saved_path = results[0].save_dir
    # print("Saved at:", saved_path)

    result_label.config(text="Detection Complete! Check 'detections/' folder.")


# Start GUI loop
root.mainloop()

adding popup window

In [ ]:
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox


# function to read the model to avoid errors in .exe files later
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best.pt")
model = YOLO(model_path)

# --- GUI setup ---
root = tk.Tk()
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x700") # size of the application window

# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path:
        file_label.config(text=f"Selected: {file_path}")
        process_file(file_path)

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None
live_running = False

# function to start live deteciton
def start_live_detection():
    global cap, live_running
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        result_label.config(text="Camera not accessible")
        return
    live_running = True
    threading.Thread(target=show_live_frame, daemon=True).start()

# function to stop detection
def stop_live_detection():
    global cap, live_running
    live_running = False
    if cap:
        cap.release()
    canvas.delete("all")
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

# display frames of camera and run model upon
def show_live_frame():
    global cap, live_running
    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            stop_live_detection()
            break

        results = model(frame)[0]

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        root.update_idletasks()
        root.update()

# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck
def clear_canvas():
    canvas.delete("all")
    canvas.image = None
    file_label.config(text="No file selected")
    result_label.config(text="")
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5)

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# file label and canvas
file_label = Label(root, text="No file selected", fg="red") # path or name of the file selected by the user, if none is selected it displays text specified
file_label.pack()

canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # Display image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        img = ImageTk.PhotoImage(img) # converting image so tkinter can display
        canvas.create_image(200, 200, anchor=tk.CENTER, image=img) # display image on canvas
        canvas.image = img # Keeps the image saved so garbage collection dont remove it

    # run the model and get results
    results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
    boxes = results[0].boxes
    num_detections = len(boxes)

    # show detection in a separate window
    results[0].show()

    # update label at the bottom of the application
    result_label.config(text="Detection Complete! Check 'detections/' folder.")

    # popup message
    messagebox.showinfo("Detection Complete",
                        f"Weapons Detected: {num_detections}\n"
                        f"Results saved in: detections/results/")

    result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen


# Start GUI loop
root.mainloop()

using canvas to display results instead of opening seperate window

In [ ]:
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox


# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best.pt")
model = YOLO(model_path)

# --- GUI setup ---
root = tk.Tk()
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x700") # size of the application window

# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path:
        file_label.config(text=f"Selected: {file_path}")
        process_file(file_path)

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None
live_running = False

# function to start live deteciton
def start_live_detection():
    global cap, live_running
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        result_label.config(text="Camera not accessible")
        return
    live_running = True
    threading.Thread(target=show_live_frame, daemon=True).start()

# function to stop detection
def stop_live_detection():
    global cap, live_running
    live_running = False
    if cap:
        cap.release()
    canvas.delete("all")
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

# display frames of camera and run model upon
def show_live_frame():
    global cap, live_running
    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            stop_live_detection()
            break

        results = model(frame)[0]

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        root.update_idletasks()
        root.update()

# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas
def clear_canvas():
    canvas.delete("all")
    canvas.image = None
    file_label.config(text="No file selected")
    result_label.config(text="")
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5)

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# file label and canvas
file_label = Label(root, text="No file selected", fg="red") # path or name of the file selected by the user, if none is selected it displays text specified
file_label.pack()

canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # Display image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
# Start GUI loop
root.mainloop()

saving of recording within the show live frame

In [ ]:
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox


recording = False
recording_writer = None
recording_start_time = None
recording_frames = []

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best.pt")
model = YOLO(model_path)

# --- GUI setup ---
root = tk.Tk() # creating the UI window
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # accessing global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
def show_live_frame():
    global cap, live_running

    from collections import deque
    import time
    import imageio

    fps = cap.get(cv2.CAP_PROP_FPS) # storing fps in a variable
    if fps <= 0 or fps > 240 or fps is None: # checking if fps are inappropriate set fps to 20
        fps = 20
    print("FPS used:", fps) # printing fps to make sure appropriate fps are being used

    buffer_seconds = 5 # setting the amount of seconds to store in buffer for recording purposes
    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []  # store the actual frames for recording
    recording = False # setting recording = false first so when we start recording we set it true
    recording_end_time = 0

    # creating a directory to store recordings
    if not os.path.exists("recordings"): 
        os.makedirs("recordings")

    # printing to see if detection actually started or not
    print("Live detection started.")

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            print("Detected label:", label)
            if "weapon" in label or "gun" in label or "knife" in label:
                print(f"Weapon Detected: {label}")
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_buffer.append(frame_rgb.copy())

        # Start or extend recording
        if Gun:
            if not recording:
                print("Starting new recording...")
                recording = True
                recording_end_time = time.time() + 5

                # Start video_frames list with buffered frames
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        # Continue recording if active
        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                print("Stopping recording.")
                recording = False

                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        # Draw boxes on live feed
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_rgb, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Show on canvas
        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

        root.update_idletasks()
        root.update()



# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

# a canvas that will display the image
canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # Display image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
# Start GUI loop
root.mainloop()

adding labels on video recordings

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox


recording = False
recording_writer = None
recording_start_time = None
recording_frames = []

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best.pt")
model = YOLO(model_path)

# --- GUI setup ---
root = tk.Tk() # creating the UI window
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
def show_live_frame():
    global cap, live_running

    from collections import deque
    import time
    import imageio

    fps = cap.get(cv2.CAP_PROP_FPS) # storing fps in a variable
    if fps <= 0 or fps > 240 or fps is None: # checking if fps are inappropriate set fps to 20
        fps = 20
    print("FPS used:", fps) # printing fps to make sure appropriate fps are being used

    buffer_seconds = 5 # setting the amount of seconds to store in buffer for recording purposes
    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []  # store the actual frames for recording
    recording = False # setting recording = false first so when we start recording we set it true
    recording_end_time = 0

    # creating a directory to store recordings
    if not os.path.exists("recordings"): 
        os.makedirs("recordings")

    # printing to see if detection actually started or not
    print("Live detection started.")

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            print("Detected label:", label)
            if "weapon" in label or "gun" in label or "knife" in label:
                print(f"Weapon Detected: {label}")
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Draw boxes on the frame before saving
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_rgb, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())

        # Start or extend recording
        if Gun:
            if not recording:
                print("Starting new recording...")
                recording = True
                recording_end_time = time.time() + 5

                # Start video_frames list with buffered frames
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        # Continue recording if active
        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                print("Stopping recording.")
                recording = False

                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        # Draw boxes on live feed
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_rgb, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Show on canvas
        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

        root.update_idletasks()
        root.update()



# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

# a canvas that will display the image
canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

adding the settings button

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox


recording = False
recording_writer = None
recording_start_time = None
recording_frames = []

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best.pt")
model = YOLO(model_path)

# --- GUI setup ---
root = tk.Tk() # creating the UI window
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

selected_fps = 25

def open_settings():
    global selected_fps

    settings_window = tk.Toplevel(root)
    settings_window.title("Settings")
    settings_window.geometry("300x200")
    
    tk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10)

    # FPS Option
    fps_frame = tk.Frame(settings_window)
    fps_frame.pack(pady=10)

    tk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = tk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    def save_settings():
        global selected_fps
        try:
            selected_fps = int(fps_spinbox.get())
            messagebox.showinfo("Settings Saved", f"Live detection FPS set to {selected_fps}")
            settings_window.destroy()
        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS value.")

    tk.Button(settings_window, text="Save", command=save_settings).pack(pady=10)
settings_button = tk.Button(root, text="⚙️", font=("Arial", 14), command=open_settings, bd=0)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
def show_live_frame():
    global cap, live_running, selected_fps

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    print("Using selected FPS:", fps)

    print("FPS used:", fps) # printing fps to make sure appropriate fps are being used

    buffer_seconds = 5 # setting the amount of seconds to store in buffer for recording purposes
    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []  # store the actual frames for recording
    recording = False # setting recording = false first so when we start recording we set it true
    recording_end_time = 0

    # creating a directory to store recordings
    if not os.path.exists("recordings"): 
        os.makedirs("recordings")

    # printing to see if detection actually started or not
    print("Live detection started.")

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            print("Detected label:", label)
            if "weapon" in label or "gun" in label or "knife" in label:
                print(f"Weapon Detected: {label}")
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Draw boxes on the frame before saving
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_rgb, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())

        # Start or extend recording
        if Gun:
            if not recording:
                print("Starting new recording...")
                recording = True
                recording_end_time = time.time() + 5

                # Start video_frames list with buffered frames
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        # Continue recording if active
        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                print("Stopping recording.")
                recording = False

                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        # Draw boxes on live feed
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_rgb, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Show on canvas
        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

        root.update_idletasks()
        root.update()



# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

# a canvas that will display the image
canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

adding datetime and fps on top

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox


recording = False
recording_writer = None
recording_start_time = None
recording_frames = []

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best.pt")
model = YOLO(model_path)

# --- GUI setup ---
root = tk.Tk() # creating the UI window
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

selected_fps = 25

def open_settings():
    global selected_fps

    settings_window = tk.Toplevel(root)
    settings_window.title("Settings")
    settings_window.geometry("300x200")
    
    tk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10)

    # FPS Option
    fps_frame = tk.Frame(settings_window)
    fps_frame.pack(pady=10)

    tk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = tk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    def save_settings():
        global selected_fps
        try:
            selected_fps = int(fps_spinbox.get())
            messagebox.showinfo("Settings Saved", f"Live detection FPS set to {selected_fps}")
            settings_window.destroy()
        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS value.")

    tk.Button(settings_window, text="Save", command=save_settings).pack(pady=10)
    tk.Button(settings_window, text="Close", command=settings_window.destroy).pack(pady=10)
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = 5
    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if "weapon" in label or "gun" in label or "knife" in label:
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_rgb, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

        root.update_idletasks()
        root.update()



# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

# a canvas that will display the image
canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

changing buttons position

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox


recording = False
recording_writer = None
recording_start_time = None
recording_frames = []

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best.pt")
model = YOLO(model_path)

# --- GUI setup ---
root = tk.Tk() # creating the UI window
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

selected_fps = 25 # setting default fps to 25 for live detection

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x200") # size of the settings window
    
    tk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = tk.Frame(settings_window)
    fps_frame.pack(pady=10)

    tk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = tk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    def save_settings():
        global selected_fps
        try:
            selected_fps = int(fps_spinbox.get())
            messagebox.showinfo("Settings Saved", f"Live detection FPS set to {selected_fps}")
            settings_window.destroy()
        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS value.")
            
    bf = tk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = 5
    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if "weapon" in label or "gun" in label or "knife" in label:
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_rgb, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

        root.update_idletasks()
        root.update()



# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

# a canvas that will display the image
canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

adding option to change amount of recording time

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox

record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best.pt")
model = YOLO(model_path)

# --- GUI setup ---
root = tk.Tk() # creating the UI window
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window


# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x200") # size of the settings window
    
    tk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = tk.Frame(settings_window)
    fps_frame.pack(pady=10)

    tk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = tk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)
    # buffer time option
    buffer_frame = tk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    tk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = tk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)


    def save_settings():
        global selected_fps
        try:
            selected_fps = int(fps_spinbox.get())
            global record_buffer_seconds
            record_buffer_seconds = int(buffer_spinbox.get())
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")
            
    bf = tk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if "weapon" in label or "gun" in label or "knife" in label:
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_rgb, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

        root.update_idletasks()
        root.update()



# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

# a canvas that will display the image
canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

fixing video detection logic

In [ ]:
detection_flag = False  # This will track whether we've already counted the weapon

for box in results.boxes:
    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(frame, label, (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Check if the weapon has already been detected in this frame or previously
    if not detection_flag:
        total_detections += 1  # Increment only once for the first frame where weapon is detected
        detection_flag = True  # Set the flag so it doesn't count again until the weapon is no longer detected
if not any(box for box in results.boxes):
    detection_flag = False  # Reset the flag when no weapon is detected


adding progress bar

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
from tkinter import ttk


record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best.pt")
model = YOLO(model_path)

# --- GUI setup ---
root = tk.Tk() # creating the UI window
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window


# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x200") # size of the settings window
    
    tk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = tk.Frame(settings_window)
    fps_frame.pack(pady=10)

    tk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = tk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)
    # buffer time option
    buffer_frame = tk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    tk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = tk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)


    def save_settings():
        global selected_fps
        try:
            selected_fps = int(fps_spinbox.get())
            global record_buffer_seconds
            record_buffer_seconds = int(buffer_spinbox.get())
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")
            
    bf = tk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        clear_canvas()
        progress.pack(pady=5)  # Show the progress bar (now above the canvas)
        progress.start(10)      # Start moving
        threading.Thread(target=process_file, args=(file_path,)).start()
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if "weapon" in label or "gun" in label or "knife" in label:
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_rgb, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)

        canvas.create_image(0, 0, anchor="nw", image=img)
        canvas.image = img

    print("Live detection stopped.")

# progress bar widget 
progress = ttk.Progressbar(root, mode='indeterminate')

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

# progress bar (moved above the canvas)
progress.pack(pady=5)  # Now appears right below the file label

# a canvas that will display the image
canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="Result will be displayed here", font=("Arial", 12), fg="blue")
result_label.pack(pady=10)

root.mainloop() # the root application window will stay running until manually closed

In [ ]:
import os
import yaml

images_dir = 'archive/weapon_detection/train/images'
labels_dir = 'archive/weapon_detection/train/labels'

data = {'images': []}

for img_file in os.listdir(images_dir):
    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        base_name = os.path.splitext(img_file)[0]
        label_file = os.path.join(labels_dir, base_name + '.txt')
        objects = []
        if os.path.exists(label_file):
            with open(label_file, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        class_id = int(parts[0])
                        bbox = list(map(float, parts[1:]))
                        objects.append({
                            'class_id': class_id,
                            'bbox': bbox
                        })
        data['images'].append({
            'path': os.path.join('images', img_file),
            'objects': objects
        })

# Save to YAML
with open('dataset.yaml', 'w') as f:
    yaml.dump(data, f, sort_keys=False)


adding themes

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
import darkdetect
from tkinter import ttk


record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
dark_mode = False
theme_preference = "System"  # "Light", "Dark", or "System" the default
is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

def get_system_theme():
    return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # Theme selection
    theme_frame = ttk.Frame(settings_window)
    theme_frame.pack(pady=10)

    ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    theme_options = ["System", "Light", "Dark"]
    theme_var = tk.StringVar()
    theme_var.set(theme_preference if theme_preference in theme_options else "System")
    theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    theme_menu.pack(side=tk.LEFT)

    def setup_ttk_styles(bg, fg):
        style = ttk.Style(settings_window)
        style.theme_use('default')  # Ensure we're using a theme we can override
        style.configure('.', background=bg, foreground=fg)
        style.configure('TLabel', background=bg, foreground=fg)
        style.configure('TButton', background=bg, foreground=fg)
        style.configure('TFrame', background=bg)
        style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
        style.configure('TMenubutton', background=bg, foreground=fg)

    def apply_theme():
        global dark_mode

        # determine theme from preference
        if theme_preference == "System":
            dark_mode = darkdetect.isDark()
        else:
            dark_mode = (theme_preference == "Dark")

        bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        fg = "#ffffff" if dark_mode else "#000000"

        root.configure(bg=bg)
        settings_button.configure(bg=bg, fg=fg)

        settings_window.configure(bg=bg)

        setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            theme_preference = theme_var.get()
            apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    ttk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    ttk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if "weapon" in label or "gun" in label or "knife" in label:
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_rgb, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

        root.update_idletasks()
        root.update()



# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

# a canvas that will display the image
canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

In [ ]:
pip install darkdetect

In [ ]:
pip install --upgrade pip

changing confidence level

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
import darkdetect
from tkinter import ttk


record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
dark_mode = False
theme_preference = "System"  # "Light", "Dark", or "System" the default
is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window
root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

def get_system_theme():
    return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # Theme selection
    theme_frame = ttk.Frame(settings_window)
    theme_frame.pack(pady=10)

    ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    theme_options = ["System", "Light", "Dark"]
    theme_var = tk.StringVar()
    theme_var.set(theme_preference if theme_preference in theme_options else "System")
    theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    theme_menu.pack(side=tk.LEFT)

    def setup_ttk_styles(bg, fg):
        style = ttk.Style(settings_window)
        style.theme_use('default')  # Ensure we're using a theme we can override
        style.configure('.', background=bg, foreground=fg)
        style.configure('TLabel', background=bg, foreground=fg)
        style.configure('TButton', background=bg, foreground=fg)
        style.configure('TFrame', background=bg)
        style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
        style.configure('TMenubutton', background=bg, foreground=fg)

    def apply_theme():
        global dark_mode

        # determine theme from preference
        if theme_preference == "System":
            dark_mode = darkdetect.isDark()
        else:
            dark_mode = (theme_preference == "Dark")

        bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        fg = "#ffffff" if dark_mode else "#000000"

        root.configure(bg=bg)
        settings_button.configure(bg=bg, fg=fg)

        settings_window.configure(bg=bg)

        setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            theme_preference = theme_var.get()
            apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    ttk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    ttk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if "pistol" in label or "assault rifle" in label or "knife" in label:
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        for box in results.boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

        root.update_idletasks()
        root.update()



# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

# a canvas that will display the image
canvas = Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

adding icon

changing button style

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
import darkdetect
from tkinter import ttk
import ttkbootstrap as tb
from ttkbootstrap.constants import *
from ttkbootstrap import PhotoImage



record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
dark_mode = False
theme_preference = "System"  # "Light", "Dark", or "System" the default
is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tb.Window(themename="darkly") # creating the UI window

# icon_image = PhotoImage(file="gun icon.png")
# root.iconphoto(False, icon_image)

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

def get_system_theme():
    return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tb.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    tb.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = tb.Frame(settings_window)
    fps_frame.pack(pady=10)

    tb.Label(fps_frame, text="Live Detection FPS: ").pack(side=tb.LEFT)

    fps_spinbox = tb.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = tb.Frame(settings_window)
    buffer_frame.pack(pady=10)

    tb.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = tb.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tb.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # Theme selection
    theme_frame = tb.Frame(settings_window)
    theme_frame.pack(pady=10)

    tb.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    theme_options = ["System", "Light", "Dark"]
    theme_var = tk.StringVar()
    theme_var.set(theme_preference if theme_preference in theme_options else "System")
    theme_menu = tb.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    theme_menu.pack(side=tb.LEFT)

    def apply_theme():
        global is_dark_mode
        is_dark_mode = darkdetect.isDark() if theme_var.get() == "System" else (theme_var.get() == "Dark")
        style.theme_use("darkly" if is_dark_mode else "flatly")

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            theme_preference = theme_var.get()
            apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()
        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter valid numbers.")

    bf = tb.Frame(settings_window)
    bf.pack(pady=10)
    tb.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tb.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

tb.Button(root, text="Select Image/Video", bootstyle="danger", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if "pistol" in label or "assault rifle" in label or "knife" in label:
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        for box in results.boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

        root.update_idletasks()
        root.update()

# button to start viewing live detection using camera, calls the start_live_detection function when pressed
tb.Button(root, text="Start Live Detection", bootstyle="danger", command=start_live_detection).pack(pady=5)

# root = tb.Window(themename="darkly")  # or "cosmo", "flatly", etc.

# Create styled button
tb.Button(root, text="Stop Live Detection", bootstyle="danger", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    result_label.config(text="") # label below the image
    

tb.Button(root, text="Clear Screen", bootstyle="danger", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
tb.Button(root, text="Exit", bootstyle="danger", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = tb.Label(root, text="No file selected", foreground="red")
file_label.pack()

# a canvas that will display the image
canvas = tb.Canvas(root, width=400, height=400, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", foreground="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

In [ ]:
pip install ttkbootstrap

custom buttons

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
import darkdetect
from tkinter import ttk
from tkinter import PhotoImage
# creating a thread safe queue to pass images to the main thread
from queue import Queue
frame_queue = Queue()



record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
dark_mode = False
theme_preference = "System"  # "Light", "Dark", or "System" the default
is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="gun icon.png")
root.iconphoto(True, icon_image)

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("700x765") # size of the application window
# root.attributes('-fullscreen', True)
root.bind("<Escape>", lambda e: root.attributes('-fullscreen', False))

def get_system_theme():
    return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # Theme selection
    theme_frame = ttk.Frame(settings_window)
    theme_frame.pack(pady=10)

    ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    theme_options = ["System", "Light", "Dark"]
    theme_var = tk.StringVar()
    theme_var.set(theme_preference if theme_preference in theme_options else "System")
    theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    theme_menu.pack(side=tk.LEFT)

    def setup_ttk_styles(bg, fg):
        style = ttk.Style(settings_window)
        style.theme_use('default')  
        style.configure('.', background=bg, foreground=fg)
        style.configure('TLabel', background=bg, foreground=fg)
        style.configure('TButton', background=bg, foreground=fg)
        style.configure('TFrame', background=bg)
        style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
        style.configure('TMenubutton', background=bg, foreground=fg)

    def apply_theme():
        global dark_mode

        # determine theme from preference
        if theme_preference == "System":
            dark_mode = darkdetect.isDark()
        else:
            dark_mode = (theme_preference == "Dark")

        bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        fg = "#ffffff" if dark_mode else "#000000"

        root.configure(bg=bg)
        settings_button.configure(bg=bg, fg=fg)

        settings_window.configure(bg=bg)

        setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            theme_preference = theme_var.get()
            apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    ttk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    ttk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

# Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer


select_btn_img = ImageTk.PhotoImage(file="select.png")

custom_button = tk.Button(root, image=select_btn_img, borderwidth=0, highlightthickness=0, command=select_file)
custom_button.image = select_btn_img
custom_button.pack()




cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start()
    update_canvas()
    # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    prev_time = time.time()

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = time.time()

        results = model(frame)[0]
        Gun = any(
            "pistol" in model.names[int(box.cls[0])].lower() or
            "assault rifle" in model.names[int(box.cls[0])].lower() or
            "knife" in model.names[int(box.cls[0])].lower()
            for box in results.boxes
        )

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        for box in results.boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        if recording:
            video_frames.append(frame_rgb.copy())
            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")
                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)
                video_frames = []

        # Push frame to queue
        if not frame_queue.full():
            frame_queue.put(frame_rgb.copy())

def update_canvas():
    if not frame_queue.empty():
        frame_rgb = frame_queue.get()
        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

    if live_running:
        root.after(10, update_canvas)


# button to start viewing live detection using camera, calls the start_live_detection function when pressed
from PIL import Image, ImageTk

# img = Image.open("Group 2.png")
# img = img.resize((100, 40), Image.Resampling.LANCZOS)  # Adjust size here
start_btn_img = ImageTk.PhotoImage(file="start.png")

start_button = tk.Button(root, image=start_btn_img, borderwidth=0, highlightthickness=0, command=start_live_detection)
start_button.image = start_btn_img
start_button.pack(pady=5)


# Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
stop_btn_img = tk.PhotoImage(file="stop.png")
stop_button = tk.Button(root, image=stop_btn_img, borderwidth=0, highlightthickness=0, command=stop_live_detection)
stop_button.image = start_btn_img
stop_button.pack(pady=5)
# Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    result_label.config(text="") # label below the image
    
# Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function


clear_btn_img = tk.PhotoImage(file="clear.png")
clear_button = tk.Button(root, image=clear_btn_img, borderwidth=0, highlightthickness=0, command=clear_canvas)
clear_button.image = clear_btn_img
clear_button.pack(pady=5)

# exit button to exit the application
# Button(root, text="Exit", command=root.destroy).pack(pady=5)

exit_btn_img = tk.PhotoImage(file="exit.png")
exit_button = tk.Button(root, image=exit_btn_img, borderwidth=0, highlightthickness=0, command=root.destroy)
exit_button.image = exit_btn_img
exit_button.pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()


# a canvas that will display the image
canvas = Canvas(root, width=450, height=450, bg="gray") # the file itself selected by the user, if none is selected then background is gray
canvas.pack(pady=10)

result_label = Label(root, text="", fg="green")
result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

In [ ]:
import tkinter as tk

def toggle_canvas_fullscreen():
    global is_canvas_fullscreen
    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        # Save original size and layout
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        fullscreen_button.place(relx=0.95, rely=0.01, anchor="ne")  # Move button to top right
    else:
        canvas.place(x=20, y=60, width=450, height=450)  # Restore original size
        fullscreen_button.place(x=30, y=20)  # Restore button position

root = tk.Tk()
root.geometry("800x600")

is_canvas_fullscreen = False

# Canvas
canvas = tk.Canvas(root, bg="gray")
canvas.place(x=20, y=60, width=450, height=450)  # Initial position and size

# Fullscreen toggle button on canvas
fullscreen_button = tk.Button(root, text="enter full screen", command=toggle_canvas_fullscreen)
fullscreen_button.place(x=30, y=20)  # Position near canvas

root.mainloop()


custom buttons, background, result label disabled, theme mode disabled

In [2]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
import darkdetect
from tkinter import ttk
from tkinter import PhotoImage
# creating a thread safe queue to pass images to the main thread
from queue import Queue
frame_queue = Queue()



record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
dark_mode = False
theme_preference = "System"  # "Light", "Dark", or "System" the default
is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="gun icon.png")
root.iconphoto(True, icon_image)


# Load background image (match window size or resize as needed)
bg_image = Image.open("BG.png")  # loading image
bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
bg_photo = ImageTk.PhotoImage(bg_image)

# Create label and place it
bg_label = tk.Label(root, image=bg_photo)
bg_label.image = bg_photo  # keep reference
bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # full window

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("700x765") # size of the application window
# root.attributes('-fullscreen', True)
root.bind("<Escape>", lambda e: root.attributes('-fullscreen', False))

def get_system_theme():
    return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # # Theme selection
    # theme_frame = ttk.Frame(settings_window)
    # theme_frame.pack(pady=10)

    # ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    # theme_options = ["System", "Light", "Dark"]
    # theme_var = tk.StringVar()
    # theme_var.set(theme_preference if theme_preference in theme_options else "System")
    # theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    # theme_menu.pack(side=tk.LEFT)

    # def setup_ttk_styles(bg, fg):
    #     style = ttk.Style(settings_window)
    #     style.theme_use('default')  
    #     style.configure('.', background=bg, foreground=fg)
    #     style.configure('TLabel', background=bg, foreground=fg)
    #     style.configure('TButton', background=bg, foreground=fg)
    #     style.configure('TFrame', background=bg)
    #     style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
    #     style.configure('TMenubutton', background=bg, foreground=fg)

    # def apply_theme():
    #     global dark_mode

    #     # determine theme from preference
    #     if theme_preference == "System":
    #         dark_mode = darkdetect.isDark()
    #     else:
    #         dark_mode = (theme_preference == "Dark")

    #     bg = "#2e2e2e" if dark_mode else "#f0f0f0"
    #     fg = "#ffffff" if dark_mode else "#000000"

    #     root.configure(bg=bg)
    #     settings_button.configure(bg=bg, fg=fg)

    #     settings_window.configure(bg=bg)

    #     setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            # theme_preference = theme_var.get()
            # apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

# Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer


select_btn_img = ImageTk.PhotoImage(file="buttons/select.png")

custom_button = tk.Button(root, image=select_btn_img, borderwidth=0, highlightthickness=0, command=select_file)
custom_button.image = select_btn_img
custom_button.pack()

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start()
    update_canvas()
    # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    prev_time = time.time()

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = time.time()

        results = model(frame)[0]
        Gun = any(
            "pistol" in model.names[int(box.cls[0])].lower() or
            "assault rifle" in model.names[int(box.cls[0])].lower() or
            "knife" in model.names[int(box.cls[0])].lower()
            for box in results.boxes
        )

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        for box in results.boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        if recording:
            video_frames.append(frame_rgb.copy())
            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")
                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)
                video_frames = []

        # Push frame to queue
        if not frame_queue.full():
            frame_queue.put(frame_rgb.copy())

def update_canvas():
    if not frame_queue.empty():
        frame_rgb = frame_queue.get()
        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

    if live_running:
        root.after(10, update_canvas)


# button to start viewing live detection using camera, calls the start_live_detection function when pressed


# img = Image.open("Group 2.png")
# img = img.resize((100, 40), Image.Resampling.LANCZOS)  # Adjust size here
start_btn_img = ImageTk.PhotoImage(file="buttons/start.png")

start_button = tk.Button(root, image=start_btn_img, borderwidth=0, highlightthickness=0, command=start_live_detection)
start_button.image = start_btn_img
start_button.pack(pady=5)


# Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
stop_btn_img = tk.PhotoImage(file="buttons/stop.png")
stop_button = tk.Button(root, image=stop_btn_img, borderwidth=0, highlightthickness=0, command=stop_live_detection)
stop_button.image = start_btn_img
stop_button.pack(pady=5)
# Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    result_label.config(text="") # label below the image
    
# Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function


clear_btn_img = tk.PhotoImage(file="buttons/clear.png")
clear_button = tk.Button(root, image=clear_btn_img, borderwidth=0, highlightthickness=0, command=clear_canvas)
clear_button.image = clear_btn_img
clear_button.pack(pady=5)

# exit button to exit the application
# Button(root, text="Exit", command=root.destroy).pack(pady=5)

exit_btn_img = tk.PhotoImage(file="buttons/exit.png")
exit_button = tk.Button(root, image=exit_btn_img, borderwidth=0, highlightthickness=0, command=root.destroy)
exit_button.image = exit_btn_img
exit_button.pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

def toggle_canvas_fullscreen():
    global is_canvas_fullscreen
    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        # Save original size and layout
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        fullscreen_button.place(relx=0.95, rely=0.01, anchor="ne")  # Move button to top right
    else:
        canvas.place(x=200, y=500, width=450, height=450)  # Restore original size
        fullscreen_button.place(relx=1, rely=1, anchor="se")  # Restore button position
    root.update_idletasks()

is_canvas_fullscreen = False
canvas_width = 450
canvas_height = 450
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 20  # 20px space below label
# a canvas that will display the image
canvas = Canvas(root, width=450, height=450, bg="gray") # the file itself selected by the user, if none is selected then background is gray
# canvas.pack(pady=10)
canvas.place(x=canvas_x, y=canvas_y, width=450, height=450, anchor="s")  # Initial position and size

# Fullscreen toggle button on canvas
fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
fullscreen_button.place(relx=0, rely=1, anchor="se")  # Position near canvas

result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
import darkdetect
from tkinter import ttk


record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
dark_mode = False
theme_preference = "System"  # "Light", "Dark", or "System" the default
is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="gun icon.png")
root.iconphoto(True, icon_image)

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

def get_system_theme():
    return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # theme selection
    # theme_frame = ttk.Frame(settings_window)
    # theme_frame.pack(pady=10)

    # ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    # theme_options = ["System", "Light", "Dark"]
    # theme_var = tk.StringVar()
    # theme_var.set(theme_preference if theme_preference in theme_options else "System")
    # theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    # theme_menu.pack(side=tk.LEFT)

    # def setup_ttk_styles(bg, fg):
    #     style = ttk.Style(settings_window)
    #     style.theme_use('default')  
    #     style.configure('.', background=bg, foreground=fg)
    #     style.configure('TLabel', background=bg, foreground=fg)
    #     style.configure('TButton', background=bg, foreground=fg)
    #     style.configure('TFrame', background=bg)
    #     style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
    #     style.configure('TMenubutton', background=bg, foreground=fg)

    # def apply_theme():
    #     global dark_mode

        # determine theme from preference
        # if theme_preference == "System":
        #     dark_mode = darkdetect.isDark()
        # else:
        #     dark_mode = (theme_preference == "Dark")

        # bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        # fg = "#ffffff" if dark_mode else "#000000"

        # root.configure(bg=bg)
        # settings_button.configure(bg=bg, fg=fg)

        # settings_window.configure(bg=bg)

        # setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            # theme_preference = theme_var.get()
            # apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    # apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    # result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="Live Detection Stopped") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))  # circular buffer to store previous frames
    video_frames = []  # frames to save for recording
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    # Create a persistent canvas image object once
    canvas.update_idletasks()
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    canvas_center_x = canvas_width // 2
    canvas_center_y = canvas_height // 2
    dummy_img = ImageTk.PhotoImage(Image.new("RGB", (400, 400)))
    canvas.image_item_id = canvas.create_image(canvas_center_x, canvas_center_y, anchor=tk.CENTER, image=dummy_img)

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if "pistol" in label or "assault rifle" in label or "knife" in label:
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert to RGB for tkinter

        for box in results.boxes:
            if box.conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())  # store latest frame in buffer

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5  # record next 5 seconds
                video_frames = list(frame_buffer)  # include previous frames
            else:
                recording_end_time = time.time() + 5  # extend recording window

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))  # save video
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # timestamp

        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  # show fps

        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))  # resize for display
        imgtk = ImageTk.PhotoImage(img)

        canvas.imgtk = imgtk  # keep a reference so image is not garbage collected
        canvas.itemconfig(canvas.image_item_id, image=imgtk)  # update image without recreating

        root.update_idletasks()  # update UI
        root.update()



# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    # result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

is_canvas_fullscreen = False

# Update layout
root.update_idletasks()

# Canvas size and position
canvas_width = 450
canvas_height = 450
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 10  # 10px gap

# Save original geometry
original_canvas_geometry = {
    "x": canvas_x,
    "y": canvas_y,
    "width": canvas_width,
    "height": canvas_height
}

# Create canvas
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="gray")
canvas.place(x=canvas_x, y=canvas_y)

# Toggle function
def toggle_canvas_fullscreen():
    global is_canvas_fullscreen

    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        fullscreen_button.place(relx=0.99, rely=0.01, anchor="ne")
        fullscreen_button.config(text="Exit Full Screen")
    else:
        # Remove relwidth/relheight by re-placing with only absolute values
        canvas.place_forget()  # Fully reset placement first
        canvas.place(
            x=original_canvas_geometry["x"],
            y=original_canvas_geometry["y"],
            width=original_canvas_geometry["width"],
            height=original_canvas_geometry["height"]
        )
        fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")
        fullscreen_button.config(text="Enable Full Screen")

    root.update_idletasks()

# Place fullscreen toggle button on root (not canvas)
fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")


# result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist

        # center the image on canvas
        canvas.update()  # make sure canvas has correct width/height
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        # center the detection image on canvas
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        # result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            # result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            # result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            # result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

making the image show on the entire canvas 

In [3]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
import darkdetect
from tkinter import ttk


record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
dark_mode = False
theme_preference = "System"  # "Light", "Dark", or "System" the default
is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="gun icon.png")
root.iconphoto(True, icon_image)

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

def get_system_theme():
    return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # theme selection
    # theme_frame = ttk.Frame(settings_window)
    # theme_frame.pack(pady=10)

    # ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    # theme_options = ["System", "Light", "Dark"]
    # theme_var = tk.StringVar()
    # theme_var.set(theme_preference if theme_preference in theme_options else "System")
    # theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    # theme_menu.pack(side=tk.LEFT)

    # def setup_ttk_styles(bg, fg):
    #     style = ttk.Style(settings_window)
    #     style.theme_use('default')  
    #     style.configure('.', background=bg, foreground=fg)
    #     style.configure('TLabel', background=bg, foreground=fg)
    #     style.configure('TButton', background=bg, foreground=fg)
    #     style.configure('TFrame', background=bg)
    #     style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
    #     style.configure('TMenubutton', background=bg, foreground=fg)

    # def apply_theme():
    #     global dark_mode

        # determine theme from preference
        # if theme_preference == "System":
        #     dark_mode = darkdetect.isDark()
        # else:
        #     dark_mode = (theme_preference == "Dark")

        # bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        # fg = "#ffffff" if dark_mode else "#000000"

        # root.configure(bg=bg)
        # settings_button.configure(bg=bg, fg=fg)

        # settings_window.configure(bg=bg)

        # setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            # theme_preference = theme_var.get()
            # apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    # apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    # result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="Live Detection Stopped") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))  # circular buffer to store previous frames
    video_frames = []  # frames to save for recording
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    # Create a persistent canvas image object once
    canvas.update_idletasks()
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    canvas_center_x = canvas_width // 2
    canvas_center_y = canvas_height // 2
    dummy_img = ImageTk.PhotoImage(Image.new("RGB", (400, 400)))
    canvas.image_item_id = canvas.create_image(canvas_center_x, canvas_center_y, anchor=tk.CENTER, image=dummy_img)

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if "pistol" in label or "assault rifle" in label or "knife" in label:
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert to RGB for tkinter

        for box in results.boxes:
            if box.conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())  # store latest frame in buffer

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5  # record next 5 seconds
                video_frames = list(frame_buffer)  # include previous frames
            else:
                recording_end_time = time.time() + 5  # extend recording window

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))  # save video
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # timestamp

        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  # show fps

        # img = Image.fromarray(frame_rgb)

        # Dynamically get current canvas size
        canvas.update_idletasks()
        canvas_width = canvas.winfo_width()
        canvas_height = canvas.winfo_height()
        
        img = Image.fromarray(frame_rgb)
       # Resize to fill the entire canvas (may stretch image slightly)
        img_resized = img.resize((canvas_width, canvas_height), Image.LANCZOS)
        imgtk = ImageTk.PhotoImage(img_resized)




        
        # img = img.resize((current_width, current_height), Image.LANCZOS)
        # imgtk = ImageTk.PhotoImage(img)


        canvas.imgtk = imgtk  # keep a reference so image is not garbage collected
        canvas.itemconfig(canvas.image_item_id, image=imgtk)
        canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)



        root.update_idletasks()  # update UI
        root.update()




# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    # result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

is_canvas_fullscreen = False

# Update layout
root.update_idletasks()

# Canvas size and position
canvas_width = 450
canvas_height = 450
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 10  # 10px gap

# Save original geometry
original_canvas_geometry = {
    "x": canvas_x,
    "y": canvas_y,
    "width": canvas_width,
    "height": canvas_height
}

# Create canvas
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="gray")
canvas.place(relx=0.5, y=canvas_y, anchor="n")

# Toggle function
def toggle_canvas_fullscreen():
    global is_canvas_fullscreen

    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        fullscreen_button.place(relx=0.99, rely=0.01, anchor="ne")
        fullscreen_button.config(text="Exit Full Screen")
    else:
        canvas.place_forget()  # Fully reset placement first

        # Center the canvas horizontally using relx
        canvas.place(
            relx=0.5,
            y=original_canvas_geometry["y"],
            anchor="n",
            width=original_canvas_geometry["width"],
            height=original_canvas_geometry["height"]
        )

        fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")
        fullscreen_button.config(text="Enable Full Screen")

    root.update_idletasks()


# Place fullscreen toggle button on root (not canvas)
fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")

# Bind Escape key to exit fullscreen
root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

root.bind("<f>", lambda event: toggle_canvas_fullscreen())

def toggle_fullscreen(event=None):
    root.attributes("-fullscreen", not root.attributes("-fullscreen"))
    
import platform
# Platform-specific fullscreen toggle
if platform.system() == "Darwin":  # macOS
    root.bind("<Command-f>", toggle_fullscreen)
else:  # Windows or Linux
    root.bind("<Control-f>", toggle_fullscreen)

# result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist

        # center the image on canvas
        canvas.update()  # make sure canvas has correct width/height
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        # center the detection image on canvas
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        # result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            # result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            # result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            # result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

adding background

In [3]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
import darkdetect
from tkinter import ttk


record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
dark_mode = False
theme_preference = "System"  # "Light", "Dark", or "System" the default
is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="gun icon.png")
root.iconphoto(True, icon_image)

from PIL import Image, ImageTk

# Load background image (match window size or resize as needed)
bg_image = Image.open("BG.png")  # loading image
bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
bg_photo = ImageTk.PhotoImage(bg_image)

# Create label and place it
bg_label = tk.Label(root, image=bg_photo)
bg_label.image = bg_photo  # keep reference
bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # full window


root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

def get_system_theme():
    return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # theme selection
    # theme_frame = ttk.Frame(settings_window)
    # theme_frame.pack(pady=10)

    # ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    # theme_options = ["System", "Light", "Dark"]
    # theme_var = tk.StringVar()
    # theme_var.set(theme_preference if theme_preference in theme_options else "System")
    # theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    # theme_menu.pack(side=tk.LEFT)

    # def setup_ttk_styles(bg, fg):
    #     style = ttk.Style(settings_window)
    #     style.theme_use('default')  
    #     style.configure('.', background=bg, foreground=fg)
    #     style.configure('TLabel', background=bg, foreground=fg)
    #     style.configure('TButton', background=bg, foreground=fg)
    #     style.configure('TFrame', background=bg)
    #     style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
    #     style.configure('TMenubutton', background=bg, foreground=fg)

    # def apply_theme():
    #     global dark_mode

        # determine theme from preference
        # if theme_preference == "System":
        #     dark_mode = darkdetect.isDark()
        # else:
        #     dark_mode = (theme_preference == "Dark")

        # bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        # fg = "#ffffff" if dark_mode else "#000000"

        # root.configure(bg=bg)
        # settings_button.configure(bg=bg, fg=fg)

        # settings_window.configure(bg=bg)

        # setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            # theme_preference = theme_var.get()
            # apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    # apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    # result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="Live Detection Stopped") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))  # circular buffer to store previous frames
    video_frames = []  # frames to save for recording
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    # Create a persistent canvas image object once
    canvas.update_idletasks()
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    canvas_center_x = canvas_width // 2
    canvas_center_y = canvas_height // 2
    dummy_img = ImageTk.PhotoImage(Image.new("RGB", (400, 400)))
    canvas.image_item_id = canvas.create_image(canvas_center_x, canvas_center_y, anchor=tk.CENTER, image=dummy_img)

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if "pistol" in label or "assault rifle" in label or "knife" in label:
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert to RGB for tkinter

        for box in results.boxes:
            if box.conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())  # store latest frame in buffer

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5  # record next 5 seconds
                video_frames = list(frame_buffer)  # include previous frames
            else:
                recording_end_time = time.time() + 5  # extend recording window

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))  # save video
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # timestamp

        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  # show fps

        # img = Image.fromarray(frame_rgb)

        # Dynamically get current canvas size
        canvas.update_idletasks()
        canvas_width = canvas.winfo_width()
        canvas_height = canvas.winfo_height()
        
        img = Image.fromarray(frame_rgb)
        # Resize to fill the entire canvas (may stretch image slightly)
        img_resized = img.resize((canvas_width, canvas_height), Image.LANCZOS)
        imgtk = ImageTk.PhotoImage(img_resized)

        # img = img.resize((current_width, current_height), Image.LANCZOS)
        # imgtk = ImageTk.PhotoImage(img)

        canvas.imgtk = imgtk
        canvas.itemconfig(canvas.image_item_id, image=imgtk)
        
        def delayed_center():
            canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)

        canvas.after(10, delayed_center)  # Allow Tk to resize canvas before positioning image


        root.update_idletasks()  # update UI
        root.update()


# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    # result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

is_canvas_fullscreen = False

# Update layout
root.update_idletasks()

# Canvas size and position
canvas_width = 450
canvas_height = 450
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 10  # 10px gap

# Save original geometry
original_canvas_geometry = {
    "x": canvas_x,
    "y": canvas_y,
    "width": canvas_width,
    "height": canvas_height
}

# Create canvas
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="gray")
canvas.place(relx=0.5, y=canvas_y, anchor="n")

def update_canvas_image_position():
    if hasattr(canvas, "image_item_id"):
        canvas.update_idletasks()
        canvas.coords(
            canvas.image_item_id,
            canvas.winfo_width() // 2,
            canvas.winfo_height() // 2
        )

def toggle_canvas_fullscreen():
    global is_canvas_fullscreen

    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        fullscreen_button.place(relx=0.99, rely=0.01, anchor="ne")
        fullscreen_button.config(text="Exit Full Screen")
    else:
        canvas.place_forget()  # remove canvas
        canvas.place(
            relx=0.5,
            y=original_canvas_geometry["y"],
            anchor="n",
            width=original_canvas_geometry["width"],
            height=original_canvas_geometry["height"]
        )
        fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")
        fullscreen_button.config(text="Enable Full Screen")

    # Force canvas update and reposition image
    root.after(100, update_canvas_image_position)




# Place fullscreen toggle button on root (not canvas)
fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")

# Bind Escape key to exit fullscreen
root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

root.bind("<f>", lambda event: toggle_canvas_fullscreen())

def toggle_fullscreen(event=None):
    root.attributes("-fullscreen", not root.attributes("-fullscreen"))
    
import platform
# Platform-specific fullscreen toggle
if platform.system() == "Darwin":  # macOS
    root.bind("<Command-f>", toggle_fullscreen)
else:  # Windows or Linux
    root.bind("<Control-f>", toggle_fullscreen)

# result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist

        # center the image on canvas
        canvas.update()  # make sure canvas has correct width/height
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        # center the detection image on canvas
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        # result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            # result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            # result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            # result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

custom button, background, theme disabled, result label disabled, issue with canvas

In [2]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
import darkdetect
from tkinter import ttk
from tkinter import PhotoImage
# creating a thread safe queue to pass images to the main thread
from queue import Queue
frame_queue = Queue()



record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
dark_mode = False
theme_preference = "System"  # "Light", "Dark", or "System" the default
is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="gun icon.png")
root.iconphoto(True, icon_image)


# Load background image (match window size or resize as needed)
bg_image = Image.open("BG.png")  # loading image
bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
bg_photo = ImageTk.PhotoImage(bg_image)

# Create label and place it
bg_label = tk.Label(root, image=bg_photo)
bg_label.image = bg_photo  # keep reference
bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # full window

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("700x765") # size of the application window
# root.attributes('-fullscreen', True)
root.bind("<Escape>", lambda e: root.attributes('-fullscreen', False))

def get_system_theme():
    return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # # Theme selection
    # theme_frame = ttk.Frame(settings_window)
    # theme_frame.pack(pady=10)

    # ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    # theme_options = ["System", "Light", "Dark"]
    # theme_var = tk.StringVar()
    # theme_var.set(theme_preference if theme_preference in theme_options else "System")
    # theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    # theme_menu.pack(side=tk.LEFT)

    # def setup_ttk_styles(bg, fg):
    #     style = ttk.Style(settings_window)
    #     style.theme_use('default')  
    #     style.configure('.', background=bg, foreground=fg)
    #     style.configure('TLabel', background=bg, foreground=fg)
    #     style.configure('TButton', background=bg, foreground=fg)
    #     style.configure('TFrame', background=bg)
    #     style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
    #     style.configure('TMenubutton', background=bg, foreground=fg)

    # def apply_theme():
    #     global dark_mode

    #     # determine theme from preference
    #     if theme_preference == "System":
    #         dark_mode = darkdetect.isDark()
    #     else:
    #         dark_mode = (theme_preference == "Dark")

    #     bg = "#2e2e2e" if dark_mode else "#f0f0f0"
    #     fg = "#ffffff" if dark_mode else "#000000"

    #     root.configure(bg=bg)
    #     settings_button.configure(bg=bg, fg=fg)

    #     settings_window.configure(bg=bg)

    #     setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            # theme_preference = theme_var.get()
            # apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

# Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer


select_btn_img = ImageTk.PhotoImage(file="buttons/select.png")

custom_button = tk.Button(root, image=select_btn_img, borderwidth=0, highlightthickness=0, command=select_file)
custom_button.image = select_btn_img
custom_button.pack()

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start()
    update_canvas()
    # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="No file selected") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))
    video_frames = []
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    prev_time = time.time()

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = time.time()

        results = model(frame)[0]
        Gun = any(
            "pistol" in model.names[int(box.cls[0])].lower() or
            "assault rifle" in model.names[int(box.cls[0])].lower() or
            "knife" in model.names[int(box.cls[0])].lower()
            for box in results.boxes
        )

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        for box in results.boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5
                video_frames = list(frame_buffer)
            else:
                recording_end_time = time.time() + 5

        if recording:
            video_frames.append(frame_rgb.copy())
            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")
                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)
                video_frames = []

        # Push frame to queue
        if not frame_queue.full():
            frame_queue.put(frame_rgb.copy())

def update_canvas():
    if not frame_queue.empty():
        frame_rgb = frame_queue.get()
        img = Image.fromarray(frame_rgb)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk

    if live_running:
        root.after(10, update_canvas)


# button to start viewing live detection using camera, calls the start_live_detection function when pressed


# img = Image.open("Group 2.png")
# img = img.resize((100, 40), Image.Resampling.LANCZOS)  # Adjust size here
start_btn_img = ImageTk.PhotoImage(file="buttons/start.png")

start_button = tk.Button(root, image=start_btn_img, borderwidth=0, highlightthickness=0, command=start_live_detection)
start_button.image = start_btn_img
start_button.pack(pady=5)


# Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
stop_btn_img = tk.PhotoImage(file="buttons/stop.png")
stop_button = tk.Button(root, image=stop_btn_img, borderwidth=0, highlightthickness=0, command=stop_live_detection)
stop_button.image = start_btn_img
stop_button.pack(pady=5)
# Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    result_label.config(text="") # label below the image
    
# Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function


clear_btn_img = tk.PhotoImage(file="buttons/clear.png")
clear_button = tk.Button(root, image=clear_btn_img, borderwidth=0, highlightthickness=0, command=clear_canvas)
clear_button.image = clear_btn_img
clear_button.pack(pady=5)

# exit button to exit the application
# Button(root, text="Exit", command=root.destroy).pack(pady=5)

exit_btn_img = tk.PhotoImage(file="buttons/exit.png")
exit_button = tk.Button(root, image=exit_btn_img, borderwidth=0, highlightthickness=0, command=root.destroy)
exit_button.image = exit_btn_img
exit_button.pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

def toggle_canvas_fullscreen():
    global is_canvas_fullscreen
    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        # Save original size and layout
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        fullscreen_button.place(relx=0.95, rely=0.01, anchor="ne")  # Move button to top right
    else:
        canvas.place(x=200, y=500, width=450, height=450)  # Restore original size
        fullscreen_button.place(relx=1, rely=1, anchor="se")  # Restore button position
    root.update_idletasks()

is_canvas_fullscreen = False
canvas_width = 450
canvas_height = 450
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 20  # 20px space below label
# a canvas that will display the image
canvas = Canvas(root, width=450, height=450, bg="gray") # the file itself selected by the user, if none is selected then background is gray
# canvas.pack(pady=10)
canvas.place(x=canvas_x, y=canvas_y, width=450, height=450, anchor="s")  # Initial position and size

# Fullscreen toggle button on canvas
fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
fullscreen_button.place(relx=0, rely=1, anchor="se")  # Position near canvas

result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist
        canvas.create_image(200, 200, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)
        canvas.create_image(200, 200, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.12/tkinter/__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/pl/m919vj0j1_z39v5fc_pyrt1w0000gn/T/ipykernel_40161/1613908895.py", line 163, in open_settings
    apply_theme()  # apply current theme immediately on opening
    ^^^^^^^^^^^
NameError: name 'apply_theme' is not defined


fixing canvas issue, with custom background, full screen button

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
import darkdetect
from tkinter import ttk


record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
dark_mode = False
theme_preference = "System"  # "Light", "Dark", or "System" the default
is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="gun icon.png")
root.iconphoto(True, icon_image)

# Load background image (match window size or resize as needed)
bg_image = Image.open("BG.png")  # loading image
bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
bg_photo = ImageTk.PhotoImage(bg_image)

# Create label and place it
bg_label = tk.Label(root, image=bg_photo)
bg_label.image = bg_photo  # keep reference
bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # full window

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

def get_system_theme():
    return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # theme selection
    # theme_frame = ttk.Frame(settings_window)
    # theme_frame.pack(pady=10)

    # ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    # theme_options = ["System", "Light", "Dark"]
    # theme_var = tk.StringVar()
    # theme_var.set(theme_preference if theme_preference in theme_options else "System")
    # theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    # theme_menu.pack(side=tk.LEFT)

    # def setup_ttk_styles(bg, fg):
    #     style = ttk.Style(settings_window)
    #     style.theme_use('default')  
    #     style.configure('.', background=bg, foreground=fg)
    #     style.configure('TLabel', background=bg, foreground=fg)
    #     style.configure('TButton', background=bg, foreground=fg)
    #     style.configure('TFrame', background=bg)
    #     style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
    #     style.configure('TMenubutton', background=bg, foreground=fg)

    # def apply_theme():
    #     global dark_mode

        # determine theme from preference
        # if theme_preference == "System":
        #     dark_mode = darkdetect.isDark()
        # else:
        #     dark_mode = (theme_preference == "Dark")

        # bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        # fg = "#ffffff" if dark_mode else "#000000"

        # root.configure(bg=bg)
        # settings_button.configure(bg=bg, fg=fg)

        # settings_window.configure(bg=bg)

        # setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            # theme_preference = theme_var.get()
            # apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    # apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    # result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="Live Detection Stopped") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))  # circular buffer to store previous frames
    video_frames = []  # frames to save for recording
    recording = False
    recording_end_time = 0

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    # Create a persistent canvas image object once
    canvas.update_idletasks()
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    canvas_center_x = canvas_width // 2
    canvas_center_y = canvas_height // 2
    dummy_img = ImageTk.PhotoImage(Image.new("RGB", (400, 400)))
    canvas.image_item_id = canvas.create_image(canvas_center_x, canvas_center_y, anchor=tk.CENTER, image=dummy_img)

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if "pistol" in label or "assault rifle" in label or "knife" in label:
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert to RGB for tkinter

        for box in results.boxes:
            if box.conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())  # store latest frame in buffer

        if Gun:
            if not recording:
                recording = True
                recording_end_time = time.time() + 5  # record next 5 seconds
                video_frames = list(frame_buffer)  # include previous frames
            else:
                recording_end_time = time.time() + 5  # extend recording window

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))  # save video
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # timestamp

        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  # show fps

        # img = Image.fromarray(frame_rgb)

        # Dynamically get current canvas size
        canvas.update_idletasks()
        canvas_width = canvas.winfo_width()
        canvas_height = canvas.winfo_height()
        
        img = Image.fromarray(frame_rgb)
       # Resize to fill the entire canvas (may stretch image slightly)
        img_resized = img.resize((canvas_width, canvas_height), Image.LANCZOS)
        imgtk = ImageTk.PhotoImage(img_resized)
  
        # img = img.resize((current_width, current_height), Image.LANCZOS)
        # imgtk = ImageTk.PhotoImage(img)

        canvas.imgtk = imgtk  # keep a reference so image is not garbage collected
        canvas.itemconfig(canvas.image_item_id, image=imgtk)
        
        canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)

        root.update_idletasks()  # update UI
        root.update()

# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    # result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

is_canvas_fullscreen = False

# Update layout
root.update_idletasks()

# Canvas size and position
canvas_width = 450
canvas_height = 450
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 10  # 10px gap

# Save original geometry
original_canvas_geometry = {
    "x": canvas_x,
    "y": canvas_y,
    "width": canvas_width,
    "height": canvas_height
}

# Create canvas
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="gray")
canvas.place(relx=0.5, y=canvas_y, anchor="n")

def center_canvas_image(event=None):
    if hasattr(canvas, "image_item_id"):
        canvas.coords(
            canvas.image_item_id,
            canvas.winfo_width() // 2,
            canvas.winfo_height() // 2
        )

canvas.bind("<Configure>", center_canvas_image)

# Toggle function
def toggle_canvas_fullscreen():
    global is_canvas_fullscreen

    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        fullscreen_button.place(relx=0.99, rely=0.01, anchor="ne")
        fullscreen_button.config(text="Exit Full Screen")
    else:
        canvas.place_forget()  # Fully reset placement first

        # Center the canvas horizontally using relx
        canvas.place(
            relx=0.5,
            y=original_canvas_geometry["y"],
            anchor="n",
            width=original_canvas_geometry["width"],
            height=original_canvas_geometry["height"]
        )

        fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")
        fullscreen_button.config(text="Enable Full Screen")

    root.update_idletasks()

    # Re-center image on canvas after it is resized
    canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)


# Place fullscreen toggle button on root (not canvas)
fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")

# Bind Escape key to exit fullscreen
root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

root.bind("<f>", lambda event: toggle_canvas_fullscreen())

def toggle_fullscreen(event=None):
    root.attributes("-fullscreen", not root.attributes("-fullscreen"))
    
import platform
# Platform-specific fullscreen toggle
if platform.system() == "Darwin":  # macOS
    root.bind("<Command-f>", toggle_fullscreen)
else:  # Windows or Linux
    root.bind("<Control-f>", toggle_fullscreen)

# result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist

        # center the image on canvas
        canvas.update()  # make sure canvas has correct width/height
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        # center the detection image on canvas
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        # result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            # result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            # result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            # result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

adding alarm

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
# import darkdetect
from tkinter import ttk
from playsound import playsound


record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
# dark_mode = False
# theme_preference = "System"  # "Light", "Dark", or "System" the default
# is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="gun icon.png")
root.iconphoto(True, icon_image)

# Load background image (match window size or resize as needed)
bg_image = Image.open("BG.png")  # loading image
bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
bg_photo = ImageTk.PhotoImage(bg_image)

# Create label and place it
bg_label = tk.Label(root, image=bg_photo)
bg_label.image = bg_photo  # keep reference
bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # full window

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

# def get_system_theme():
    # return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # theme selection
    # theme_frame = ttk.Frame(settings_window)
    # theme_frame.pack(pady=10)

    # ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    # theme_options = ["System", "Light", "Dark"]
    # theme_var = tk.StringVar()
    # theme_var.set(theme_preference if theme_preference in theme_options else "System")
    # theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    # theme_menu.pack(side=tk.LEFT)

    # def setup_ttk_styles(bg, fg):
    #     style = ttk.Style(settings_window)
    #     style.theme_use('default')  
    #     style.configure('.', background=bg, foreground=fg)
    #     style.configure('TLabel', background=bg, foreground=fg)
    #     style.configure('TButton', background=bg, foreground=fg)
    #     style.configure('TFrame', background=bg)
    #     style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
    #     style.configure('TMenubutton', background=bg, foreground=fg)

    # def apply_theme():
    #     global dark_mode

        # determine theme from preference
        # if theme_preference == "System":
        #     dark_mode = darkdetect.isDark()
        # else:
        #     dark_mode = (theme_preference == "Dark")

        # bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        # fg = "#ffffff" if dark_mode else "#000000"

        # root.configure(bg=bg)
        # settings_button.configure(bg=bg, fg=fg)

        # settings_window.configure(bg=bg)

        # setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            # theme_preference = theme_var.get()
            # apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    # apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    # result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="Live Detection Stopped") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio
    # import threading
    # from playsound import playsound

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))  # circular buffer to store previous frames
    video_frames = []  # frames to save for recording
    recording = False
    recording_end_time = 0
    last_alert_time = -float("inf")


    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    # Create a persistent canvas image object once
    canvas.update_idletasks()
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    canvas_center_x = canvas_width // 2
    canvas_center_y = canvas_height // 2
    dummy_img = ImageTk.PhotoImage(Image.new("RGB", (400, 400)))
    canvas.image_item_id = canvas.create_image(canvas_center_x, canvas_center_y, anchor=tk.CENTER, image=dummy_img)

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        # alarm and recording will be done only for these weapons
        weapon_keywords = ("pistol", "assault rifle", "revolver", "shotgun")

        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if any(keyword in label for keyword in weapon_keywords):
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert to RGB for tkinter

        for box in results.boxes:
            if box.conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())  # store latest frame in buffer

        if Gun:
            # here 5 second means alarm will sound after 5 seconds of previous alarm
            if time.time() - last_alert_time > 5:
                threading.Thread(target=lambda: playsound("siren alert.mp3"), daemon=True).start()
                last_alert_time = time.time()

            if not recording:
                recording = True
                recording_end_time = time.time() + 5  # record next 5 seconds
                video_frames = list(frame_buffer)  # include previous frames
            else:
                recording_end_time = time.time() + 5  # extend recording window

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))  # save video
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # timestamp

        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  # show fps

        # img = Image.fromarray(frame_rgb)

        # Dynamically get current canvas size
        canvas.update_idletasks()
        canvas_width = canvas.winfo_width()
        canvas_height = canvas.winfo_height()
        
        img = Image.fromarray(frame_rgb)
       # Resize to fill the entire canvas (may stretch image slightly)
        img_resized = img.resize((canvas_width, canvas_height), Image.LANCZOS)
        imgtk = ImageTk.PhotoImage(img_resized)

        # img = img.resize((current_width, current_height), Image.LANCZOS)
        # imgtk = ImageTk.PhotoImage(img)

        canvas.imgtk = imgtk  # keep a reference so image is not garbage collected
        canvas.itemconfig(canvas.image_item_id, image=imgtk)

        canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)

        root.update_idletasks()  # update UI
        root.update()


# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    # result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

is_canvas_fullscreen = False

# Update layout
root.update_idletasks()

# Canvas size and position
canvas_width = 450
canvas_height = 450
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 10  # 10px gap

# Save original geometry
original_canvas_geometry = {
    "x": canvas_x,
    "y": canvas_y,
    "width": canvas_width,
    "height": canvas_height
}

# Create canvas
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="gray")
canvas.place(relx=0.5, y=canvas_y, anchor="n")

def center_canvas_image(event=None):
    if hasattr(canvas, "image_item_id"):
        canvas.coords(
            canvas.image_item_id,
            canvas.winfo_width() // 2,
            canvas.winfo_height() // 2
        )

canvas.bind("<Configure>", center_canvas_image)

# Toggle function
def toggle_canvas_fullscreen():
    global is_canvas_fullscreen

    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        fullscreen_button.place(relx=0.99, rely=0.01, anchor="ne")
        fullscreen_button.config(text="Exit Full Screen")
    else:
        canvas.place_forget()  # Fully reset placement first

        # Center the canvas horizontally using relx
        canvas.place(
            relx=0.5,
            y=original_canvas_geometry["y"],
            anchor="n",
            width=original_canvas_geometry["width"],
            height=original_canvas_geometry["height"]
        )

        fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")
        fullscreen_button.config(text="Enable Full Screen")

    root.update_idletasks()

    # Re-center image on canvas after it is resized
    canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)


# Place fullscreen toggle button on root (not canvas)
fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")

# Bind Escape key to exit fullscreen
root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

root.bind("<f>", lambda event: toggle_canvas_fullscreen())

def toggle_fullscreen(event=None):
    root.attributes("-fullscreen", not root.attributes("-fullscreen"))
    
import platform
# Platform-specific fullscreen toggle
if platform.system() == "Darwin":  # macOS
    root.bind("<Command-f>", toggle_fullscreen)
else:  # Windows or Linux
    root.bind("<Control-f>", toggle_fullscreen)

# result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist

        # center the image on canvas
        canvas.update()  # make sure canvas has correct width/height
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        # center the detection image on canvas
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        # result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            # result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            # result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            # result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

changing button layout

In [ ]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
# import darkdetect
from tkinter import ttk
from playsound import playsound


record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
# dark_mode = False
# theme_preference = "System"  # "Light", "Dark", or "System" the default
# is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="gun icon.png")
root.iconphoto(True, icon_image)

# Load background image (match window size or resize as needed)
bg_image = Image.open("BG.png")  # loading image
bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
bg_photo = ImageTk.PhotoImage(bg_image)

# Create label and place it
bg_label = tk.Label(root, image=bg_photo)
bg_label.image = bg_photo  # keep reference
bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # full window

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

# def get_system_theme():
    # return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # theme selection
    # theme_frame = ttk.Frame(settings_window)
    # theme_frame.pack(pady=10)

    # ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    # theme_options = ["System", "Light", "Dark"]
    # theme_var = tk.StringVar()
    # theme_var.set(theme_preference if theme_preference in theme_options else "System")
    # theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    # theme_menu.pack(side=tk.LEFT)

    # def setup_ttk_styles(bg, fg):
    #     style = ttk.Style(settings_window)
    #     style.theme_use('default')  
    #     style.configure('.', background=bg, foreground=fg)
    #     style.configure('TLabel', background=bg, foreground=fg)
    #     style.configure('TButton', background=bg, foreground=fg)
    #     style.configure('TFrame', background=bg)
    #     style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
    #     style.configure('TMenubutton', background=bg, foreground=fg)

    # def apply_theme():
    #     global dark_mode

        # determine theme from preference
        # if theme_preference == "System":
        #     dark_mode = darkdetect.isDark()
        # else:
        #     dark_mode = (theme_preference == "Dark")

        # bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        # fg = "#ffffff" if dark_mode else "#000000"

        # root.configure(bg=bg)
        # settings_button.configure(bg=bg, fg=fg)

        # settings_window.configure(bg=bg)

        # setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            # theme_preference = theme_var.get()
            # apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    # apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    # result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="Live Detection Stopped") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio
    # import threading
    # from playsound import playsound

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))  # circular buffer to store previous frames
    video_frames = []  # frames to save for recording
    recording = False
    recording_end_time = 0
    last_alert_time = -float("inf")


    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    # Create a persistent canvas image object once
    canvas.update_idletasks()
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    canvas_center_x = canvas_width // 2
    canvas_center_y = canvas_height // 2
    dummy_img = ImageTk.PhotoImage(Image.new("RGB", (400, 400)))
    canvas.image_item_id = canvas.create_image(canvas_center_x, canvas_center_y, anchor=tk.CENTER, image=dummy_img)

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        # alarm and recording will be done only for these weapons
        weapon_keywords = ("pistol", "assault rifle", "revolver", "shotgun")

        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if any(keyword in label for keyword in weapon_keywords):
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert to RGB for tkinter

        for box in results.boxes:
            if box.conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())  # store latest frame in buffer

        if Gun:
            # here 5 second means alarm will sound after 5 seconds of previous alarm
            if time.time() - last_alert_time > 5:
                threading.Thread(target=lambda: playsound("siren alert.mp3"), daemon=True).start()
                last_alert_time = time.time()

            if not recording:
                recording = True
                recording_end_time = time.time() + 5  # record next 5 seconds
                video_frames = list(frame_buffer)  # include previous frames
            else:
                recording_end_time = time.time() + 5  # extend recording window

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))  # save video
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # timestamp

        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  # show fps

        # img = Image.fromarray(frame_rgb)

        # Dynamically get current canvas size
        canvas.update_idletasks()
        canvas_width = canvas.winfo_width()
        canvas_height = canvas.winfo_height()
        
        img = Image.fromarray(frame_rgb)
       # Resize to fill the entire canvas (may stretch image slightly)
        img_resized = img.resize((canvas_width, canvas_height), Image.LANCZOS)
        imgtk = ImageTk.PhotoImage(img_resized)

        # img = img.resize((current_width, current_height), Image.LANCZOS)
        # imgtk = ImageTk.PhotoImage(img)

        canvas.imgtk = imgtk  # keep a reference so image is not garbage collected
        canvas.itemconfig(canvas.image_item_id, image=imgtk)

        canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)

        root.update_idletasks()  # update UI
        root.update()

bframe = ttk.Frame(root)
bframe.pack(pady=10)
tk.Button(bframe, text="Start Live Detection", command=start_live_detection).pack(side="left", padx=5)
tk.Button(bframe, text="Stop Live Detection", command=stop_live_detection).pack(side="left", padx=5)
# button to start viewing live detection using camera, calls the start_live_detection function when pressed
# Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
# Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    # result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

is_canvas_fullscreen = False

# Update layout
root.update_idletasks()

# Canvas size and position
canvas_width = 450
canvas_height = 450
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 10  # 10px gap

# Save original geometry
original_canvas_geometry = {
    "x": canvas_x,
    "y": canvas_y,
    "width": canvas_width,
    "height": canvas_height
}

# Create canvas
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="gray")
canvas.place(relx=0.5, y=canvas_y, anchor="n")

def center_canvas_image(event=None):
    if hasattr(canvas, "image_item_id"):
        canvas.coords(
            canvas.image_item_id,
            canvas.winfo_width() // 2,
            canvas.winfo_height() // 2
        )

canvas.bind("<Configure>", center_canvas_image)

# Toggle function
def toggle_canvas_fullscreen():
    global is_canvas_fullscreen

    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        fullscreen_button.place(relx=0.99, rely=0.01, anchor="ne")
        fullscreen_button.config(text="Exit Full Screen")
    else:
        canvas.place_forget()  # Fully reset placement first

        # Center the canvas horizontally using relx
        canvas.place(
            relx=0.5,
            y=original_canvas_geometry["y"],
            anchor="n",
            width=original_canvas_geometry["width"],
            height=original_canvas_geometry["height"]
        )

        fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")
        fullscreen_button.config(text="Enable Full Screen")

    root.update_idletasks()

    # Re-center image on canvas after it is resized
    canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)


# Place fullscreen toggle button on root (not canvas)
fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")

# Bind Escape key to exit fullscreen
root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

root.bind("<f>", lambda event: toggle_canvas_fullscreen())

def toggle_fullscreen(event=None):
    root.attributes("-fullscreen", not root.attributes("-fullscreen"))
    
import platform
# Platform-specific fullscreen toggle
if platform.system() == "Darwin":  # macOS
    root.bind("<Command-f>", toggle_fullscreen)
else:  # Windows or Linux
    root.bind("<Control-f>", toggle_fullscreen)

# result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist

        # center the image on canvas
        canvas.update()  # make sure canvas has correct width/height
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        # center the detection image on canvas
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        # result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            # result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            # result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            # result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

adding stop sound button

In [3]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
# import darkdetect
from tkinter import ttk
# from playsound import playsound
import pygame


record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
# dark_mode = False
# theme_preference = "System"  # "Light", "Dark", or "System" the default
# is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="gun icon.png")
root.iconphoto(True, icon_image)

# Load background image (match window size or resize as needed)
bg_image = Image.open("BG.png")  # loading image
bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
bg_photo = ImageTk.PhotoImage(bg_image)

# Create label and place it
bg_label = tk.Label(root, image=bg_photo)
bg_label.image = bg_photo  # keep reference
bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # full window

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

# def get_system_theme():
    # return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # theme selection
    # theme_frame = ttk.Frame(settings_window)
    # theme_frame.pack(pady=10)

    # ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    # theme_options = ["System", "Light", "Dark"]
    # theme_var = tk.StringVar()
    # theme_var.set(theme_preference if theme_preference in theme_options else "System")
    # theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    # theme_menu.pack(side=tk.LEFT)

    # def setup_ttk_styles(bg, fg):
    #     style = ttk.Style(settings_window)
    #     style.theme_use('default')  
    #     style.configure('.', background=bg, foreground=fg)
    #     style.configure('TLabel', background=bg, foreground=fg)
    #     style.configure('TButton', background=bg, foreground=fg)
    #     style.configure('TFrame', background=bg)
    #     style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
    #     style.configure('TMenubutton', background=bg, foreground=fg)

    # def apply_theme():
    #     global dark_mode

        # determine theme from preference
        # if theme_preference == "System":
        #     dark_mode = darkdetect.isDark()
        # else:
        #     dark_mode = (theme_preference == "Dark")

        # bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        # fg = "#ffffff" if dark_mode else "#000000"

        # root.configure(bg=bg)
        # settings_button.configure(bg=bg, fg=fg)

        # settings_window.configure(bg=bg)

        # setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            # theme_preference = theme_var.get()
            # apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    # apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    # result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="Live Detection Stopped") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime


def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio
    import threading
    # from playsound import playsound

    pygame.mixer.init()  # initialize pygame mixer

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))  # circular buffer to store previous frames
    video_frames = []  # frames to save for recording
    recording = False
    recording_end_time = 0
    last_alert_time = -float("inf")


    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    # Create a persistent canvas image object once
    canvas.update_idletasks()
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    canvas_center_x = canvas_width // 2
    canvas_center_y = canvas_height // 2
    dummy_img = ImageTk.PhotoImage(Image.new("RGB", (400, 400)))
    canvas.image_item_id = canvas.create_image(canvas_center_x, canvas_center_y, anchor=tk.CENTER, image=dummy_img)

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        weapon_keywords = ("pistol", "assault rifle", "revolver", "shotgun")

        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if any(keyword in label for keyword in weapon_keywords):
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert to RGB for tkinter

        for box in results.boxes:
            if box.conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())  # store latest frame in buffer

        if Gun:
            # here 10 second means alarm will sound after 10 seconds of previous alarm
            if time.time() - last_alert_time > 5:
                def play_alert():
                    try:
                        pygame.mixer.music.load("siren alert.mp3")
                        pygame.mixer.music.play()
                    except Exception as e:
                        print("[ERROR] Playing sound:", e)

                threading.Thread(target=play_alert, daemon=True).start()
                last_alert_time = time.time()

            if not recording:
                recording = True
                recording_end_time = time.time() + 5  # record next 5 seconds
                video_frames = list(frame_buffer)  # include previous frames
            else:
                recording_end_time = time.time() + 5  # extend recording window

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))  # save video
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                pygame.mixer.music.stop()  # stop the siren after recording ends
                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # timestamp

        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  # show fps

        # img = Image.fromarray(frame_rgb)

        # Dynamically get current canvas size
        canvas.update_idletasks()
        canvas_width = canvas.winfo_width()
        canvas_height = canvas.winfo_height()
        
        img = Image.fromarray(frame_rgb)
       # Resize to fill the entire canvas (may stretch image slightly)
        img_resized = img.resize((canvas_width, canvas_height), Image.LANCZOS)
        imgtk = ImageTk.PhotoImage(img_resized)

        # img = img.resize((current_width, current_height), Image.LANCZOS)
        # imgtk = ImageTk.PhotoImage(img)

        canvas.imgtk = imgtk  # keep a reference so image is not garbage collected
        canvas.itemconfig(canvas.image_item_id, image=imgtk)

        canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)

        root.update_idletasks()  # update UI
        root.update()



# button to start viewing live detection using camera, calls the start_live_detection function when pressed
Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

stop_sound = tk.Button(root, text="Stop Sound", command=lambda: pygame.mixer.music.stop())
stop_sound.pack(side=tk.RIGHT, anchor="e", padx=1, pady=5)

root.bind("<m>", lambda event: pygame.mixer.music.stop())


# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    # result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

is_canvas_fullscreen = False

# Update layout
root.update_idletasks()

# Canvas size and position
canvas_width = 450
canvas_height = 450
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 10  # 10px gap

# Save original geometry
original_canvas_geometry = {
    "x": canvas_x,
    "y": canvas_y,
    "width": canvas_width,
    "height": canvas_height
}

# Create canvas
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="gray")
canvas.place(relx=0.5, y=canvas_y, anchor="n")

def center_canvas_image(event=None):
    if hasattr(canvas, "image_item_id"):
        canvas.coords(
            canvas.image_item_id,
            canvas.winfo_width() // 2,
            canvas.winfo_height() // 2
        )

canvas.bind("<Configure>", center_canvas_image)

# Toggle function
def toggle_canvas_fullscreen():
    global is_canvas_fullscreen

    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        fullscreen_button.place(relx=0.99, rely=0.01, anchor="se")
        fullscreen_button.config(text="Exit Full Screen")
    else:
        canvas.place_forget()  # Fully reset placement first

        # Center the canvas horizontally using relx
        canvas.place(
            relx=0.5,
            y=original_canvas_geometry["y"],
            anchor="n",
            width=original_canvas_geometry["width"],
            height=original_canvas_geometry["height"]
        )

        fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")
        fullscreen_button.config(text="Enable Full Screen")

    root.update_idletasks()

    # Re-center image on canvas after it is resized
    canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)


# Place fullscreen toggle button on root (not canvas)
fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")

# Bind Escape key to exit fullscreen
root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

root.bind("<f>", lambda event: toggle_canvas_fullscreen())

def toggle_fullscreen(event=None):
    root.attributes("-fullscreen", not root.attributes("-fullscreen"))
    
import platform
# Platform-specific fullscreen toggle
if platform.system() == "Darwin":  # macOS
    root.bind("<Command-f>", toggle_fullscreen)
else:  # Windows or Linux
    root.bind("<Control-f>", toggle_fullscreen)

# result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist

        # center the image on canvas
        canvas.update()  # make sure canvas has correct width/height
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        # center the detection image on canvas
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        # result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            # result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            # result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            # result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

changing button positions

In [2]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
# import darkdetect
from tkinter import ttk
# from playsound import playsound
import pygame

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection
record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
# dark_mode = False
# theme_preference = "System"  # "Light", "Dark", or "System" the default
# is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)

# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="gun icon.png")
root.iconphoto(True, icon_image)

# Load background image (match window size or resize as needed)
bg_image = Image.open("BG.png")  # loading image
bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
bg_photo = ImageTk.PhotoImage(bg_image)

# Create label and place it
bg_label = tk.Label(root, image=bg_photo)
bg_label.image = bg_photo  # keep reference
bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # full window

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

# def get_system_theme():
    # return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # theme selection
    # theme_frame = ttk.Frame(settings_window)
    # theme_frame.pack(pady=10)

    # ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    # theme_options = ["System", "Light", "Dark"]
    # theme_var = tk.StringVar()
    # theme_var.set(theme_preference if theme_preference in theme_options else "System")
    # theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    # theme_menu.pack(side=tk.LEFT)

    # def setup_ttk_styles(bg, fg):
    #     style = ttk.Style(settings_window)
    #     style.theme_use('default')  
    #     style.configure('.', background=bg, foreground=fg)
    #     style.configure('TLabel', background=bg, foreground=fg)
    #     style.configure('TButton', background=bg, foreground=fg)
    #     style.configure('TFrame', background=bg)
    #     style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
    #     style.configure('TMenubutton', background=bg, foreground=fg)

    # def apply_theme():
    #     global dark_mode

        # determine theme from preference
        # if theme_preference == "System":
        #     dark_mode = darkdetect.isDark()
        # else:
        #     dark_mode = (theme_preference == "Dark")

        # bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        # fg = "#ffffff" if dark_mode else "#000000"

        # root.configure(bg=bg)
        # settings_button.configure(bg=bg, fg=fg)

        # settings_window.configure(bg=bg)

        # setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            # theme_preference = theme_var.get()
            # apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    # apply_theme()  # apply current theme immediately on opening

    
settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
# Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 


# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    # result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="Live Detection Stopped") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime


def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio
    import threading
    # from playsound import playsound

    pygame.mixer.init()  # initialize pygame mixer

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))  # circular buffer to store previous frames
    video_frames = []  # frames to save for recording
    recording = False
    recording_end_time = 0
    last_alert_time = -float("inf")


    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    # Create a persistent canvas image object once
    canvas.update_idletasks()
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    canvas_center_x = canvas_width // 2
    canvas_center_y = canvas_height // 2
    dummy_img = ImageTk.PhotoImage(Image.new("RGB", (400, 400)))
    canvas.image_item_id = canvas.create_image(canvas_center_x, canvas_center_y, anchor=tk.CENTER, image=dummy_img)

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        weapon_keywords = ("pistol", "assault rifle", "revolver", "shotgun")

        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if any(keyword in label for keyword in weapon_keywords):
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert to RGB for tkinter

        for box in results.boxes:
            if box.conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())  # store latest frame in buffer

        if Gun:
            # here 10 second means alarm will sound after 10 seconds of previous alarm
            if time.time() - last_alert_time > 5:
                def play_alert():
                    try:
                        pygame.mixer.music.load("siren alert.mp3")
                        pygame.mixer.music.play()
                    except Exception as e:
                        print("[ERROR] Playing sound:", e)

                threading.Thread(target=play_alert, daemon=True).start()
                last_alert_time = time.time()

            if not recording:
                recording = True
                recording_end_time = time.time() + 5  # record next 5 seconds
                video_frames = list(frame_buffer)  # include previous frames
            else:
                recording_end_time = time.time() + 5  # extend recording window

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))  # save video
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                pygame.mixer.music.stop()  # stop the siren after recording ends
                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # timestamp

        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  # show fps

        # img = Image.fromarray(frame_rgb)

        # Dynamically get current canvas size
        canvas.update_idletasks()
        canvas_width = canvas.winfo_width()
        canvas_height = canvas.winfo_height()
        
        img = Image.fromarray(frame_rgb)
       # Resize to fill the entire canvas (may stretch image slightly)
        img_resized = img.resize((canvas_width, canvas_height), Image.LANCZOS)
        imgtk = ImageTk.PhotoImage(img_resized)

        # img = img.resize((current_width, current_height), Image.LANCZOS)
        # imgtk = ImageTk.PhotoImage(img)

        canvas.imgtk = imgtk  # keep a reference so image is not garbage collected
        canvas.itemconfig(canvas.image_item_id, image=imgtk)

        canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)

        root.update_idletasks()  # update UI
        root.update()

# frame to group buttons to the left side
left_frame = tk.Frame(root)
left_frame.pack(side=tk.LEFT, anchor="n", padx=10, pady=10)

# button to select an image or a video from the computer
select_btn_img = tk.PhotoImage(file="new buttons/select.png")
select_button = tk.Button(left_frame, image=select_btn_img, borderwidth=0, highlightthickness=0, command=select_file)
select_button.image = select_btn_img
select_button.pack(pady=5)

# Button(left_frame, text="Select Image/Video", command=select_file).pack(pady=5) 

# button to start live detection, calls the start_live_detection function when pressed
start_btn_img = tk.PhotoImage(file="new buttons/start.png")
start_button = tk.Button(left_frame, image=start_btn_img, borderwidth=0, highlightthickness=0, command=start_live_detection)
start_button.image = start_btn_img
start_button.pack(pady=5)

# Button(left_frame, text="Start Live Detection", command=start_live_detection).pack(pady=5)


# button to stop live detection, calls the stop_live_detection function when pressed
stop_btn_img = tk.PhotoImage(file="new buttons/stop.png")
stop_button = tk.Button(left_frame, image=stop_btn_img, borderwidth=0, highlightthickness=0, command=stop_live_detection)
stop_button.image = stop_btn_img
stop_button.pack(pady=5)

# Button(left_frame, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

stop_sound = tk.Button(root, text="Stop Sound", command=lambda: pygame.mixer.music.stop())
stop_sound.pack(side=tk.RIGHT, anchor="e", padx=1, pady=5)

root.bind("<m>", lambda event: pygame.mixer.music.stop())


# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    # result_label.config(text="") # label below the image
    
Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

is_canvas_fullscreen = False

# Update layout
root.update_idletasks()

# Canvas size and position
canvas_width = 450
canvas_height = 450
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 10  # 10px gap

# Save original geometry
original_canvas_geometry = {
    "x": canvas_x,
    "y": canvas_y,
    "width": canvas_width,
    "height": canvas_height
}

# Create canvas and place it at the bottom center
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="gray")
canvas.place(relx=0.5, y=root.winfo_height() - canvas_height // 2 - 10, anchor="s")

# Function to center image on canvas
def center_canvas_image(event=None):
    if hasattr(canvas, "image_item_id"):
        canvas.coords(
            canvas.image_item_id,
            canvas.winfo_width() // 2,
            canvas.winfo_height() // 2
        )

canvas.bind("<Configure>", center_canvas_image)

# Toggle function
def toggle_canvas_fullscreen():
    global is_canvas_fullscreen

    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        fullscreen_button.place(relx=0.99, rely=0.01, anchor="se")
        fullscreen_button.config(text="Exit Full Screen")
    else:
        canvas.place_forget()  # Fully reset placement first

        # Center the canvas horizontally using relx
        canvas.place(
            relx=0.5,
            y=original_canvas_geometry["y"],
            anchor="n",
            width=original_canvas_geometry["width"],
            height=original_canvas_geometry["height"]
        )

        fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")
        fullscreen_button.config(text="Enable Full Screen")

    root.update_idletasks()

    # Re-center image on canvas after it is resized
    canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)


# Place fullscreen toggle button on root (not canvas)
fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")

# Bind Escape key to exit fullscreen
root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

root.bind("<f>", lambda event: toggle_canvas_fullscreen())

def toggle_fullscreen(event=None):
    root.attributes("-fullscreen", not root.attributes("-fullscreen"))
    
import platform
# Platform-specific fullscreen toggle
if platform.system() == "Darwin":  # macOS
    root.bind("<Command-f>", toggle_fullscreen)
else:  # Windows or Linux
    root.bind("<Control-f>", toggle_fullscreen)

# result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist

        # center the image on canvas
        canvas.update()  # make sure canvas has correct width/height
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        # center the detection image on canvas
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        # result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            # result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            # result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            # result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

checking bg and btn

In [5]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
# import darkdetect
from tkinter import ttk
from playsound import playsound


record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
# dark_mode = False
# theme_preference = "System"  # "Light", "Dark", or "System" the default
# is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)


# Tooltip class for hover mouse effect on info and help buttons
class ToolTip:
    def __init__(self, widget, text):
        self.widget = widget
        self.text = text
        self.tip_window = None
        widget.bind("<Enter>", self.show_tip)
        widget.bind("<Leave>", self.hide_tip)

    def show_tip(self, event=None):
        if self.tip_window or not self.text:
            return
        x, y, cx, cy = self.widget.bbox("insert")
        x = x + self.widget.winfo_rootx() + 25
        y = y + self.widget.winfo_rooty() + 20

        self.tip_window = tw = tk.Toplevel(self.widget)
        tw.wm_overrideredirect(True)  # Remove window decorations
        tw.wm_geometry(f"+{x}+{y}")

        label = tk.Label(tw, text=self.text, background="#ffffe0",
                         relief=tk.SOLID, borderwidth=1,
                         font=("tahoma", "8", "normal"))
        label.pack(ipadx=1)

    def hide_tip(self, event=None):
        if self.tip_window:
            self.tip_window.destroy()
            self.tip_window = None

# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="gun icon.png")
root.iconphoto(True, icon_image)

# Load background image (match window size or resize as needed)
bg_image = Image.open("BG.png")  # loading image
bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
bg_photo = ImageTk.PhotoImage(bg_image)

# Create label and place it
bg_label = tk.Label(root, image=bg_photo)
bg_label.image = bg_photo  # keep reference
bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # full window

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x705") # size of the application window

# def get_system_theme():
    # return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # theme selection
    # theme_frame = ttk.Frame(settings_window)
    # theme_frame.pack(pady=10)

    # ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    # theme_options = ["System", "Light", "Dark"]
    # theme_var = tk.StringVar()
    # theme_var.set(theme_preference if theme_preference in theme_options else "System")
    # theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    # theme_menu.pack(side=tk.LEFT)

    # def setup_ttk_styles(bg, fg):
    #     style = ttk.Style(settings_window)
    #     style.theme_use('default')  
    #     style.configure('.', background=bg, foreground=fg)
    #     style.configure('TLabel', background=bg, foreground=fg)
    #     style.configure('TButton', background=bg, foreground=fg)
    #     style.configure('TFrame', background=bg)
    #     style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
    #     style.configure('TMenubutton', background=bg, foreground=fg)

    # def apply_theme():
    #     global dark_mode

        # determine theme from preference
        # if theme_preference == "System":
        #     dark_mode = darkdetect.isDark()
        # else:
        #     dark_mode = (theme_preference == "Dark")

        # bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        # fg = "#ffffff" if dark_mode else "#000000"

        # root.configure(bg=bg)
        # settings_button.configure(bg=bg, fg=fg)

        # settings_window.configure(bg=bg)

        # setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            # theme_preference = theme_var.get()
            # apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    # apply_theme()  # apply current theme immediately on opening



settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
# Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 

Button(root, text="Select Image/Video", command=select_file).pack(pady=5) # button to select an image or a video from the computer

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection

# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    # result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="Live Detection Stopped") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime

def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio
    # import threading
    # from playsound import playsound

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))  # circular buffer to store previous frames
    video_frames = []  # frames to save for recording
    recording = False
    recording_end_time = 0
    last_alert_time = -float("inf")


    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    # Create a persistent canvas image object once
    canvas.update_idletasks()
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    canvas_center_x = canvas_width // 2
    canvas_center_y = canvas_height // 2
    dummy_img = ImageTk.PhotoImage(Image.new("RGB", (400, 400)))
    canvas.image_item_id = canvas.create_image(canvas_center_x, canvas_center_y, anchor=tk.CENTER, image=dummy_img)

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        weapon_keywords = ("pistol", "assault rifle", "revolver", "shotgun")

        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if any(keyword in label for keyword in weapon_keywords):
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert to RGB for tkinter

        for box in results.boxes:
            if box.conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())  # store latest frame in buffer

        if Gun:
            # here 10 second means alarm will sound after 10 seconds of previous alarm
            if time.time() - last_alert_time > 5:
                threading.Thread(target=lambda: playsound("siren alert.mp3"), daemon=True).start()
                last_alert_time = time.time()

            if not recording:
                recording = True
                recording_end_time = time.time() + 5  # record next 5 seconds
                video_frames = list(frame_buffer)  # include previous frames
            else:
                recording_end_time = time.time() + 5  # extend recording window

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))  # save video
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # timestamp

        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  # show fps

        # img = Image.fromarray(frame_rgb)

        # Dynamically get current canvas size
        canvas.update_idletasks()
        canvas_width = canvas.winfo_width()
        canvas_height = canvas.winfo_height()
        
        img = Image.fromarray(frame_rgb)
       # Resize to fill the entire canvas (may stretch image slightly)
        img_resized = img.resize((canvas_width, canvas_height), Image.LANCZOS)
        imgtk = ImageTk.PhotoImage(img_resized)

        # img = img.resize((current_width, current_height), Image.LANCZOS)
        # imgtk = ImageTk.PhotoImage(img)

        canvas.imgtk = imgtk  # keep a reference so image is not garbage collected
        canvas.itemconfig(canvas.image_item_id, image=imgtk)

        canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)

        root.update_idletasks()  # update UI
        root.update()


# button to start viewing live detection using camera, calls the start_live_detection function when pressed
start_btn_img = tk.PhotoImage(file="new buttons/start.png")
start_button = tk.Button(root, image=start_btn_img, borderwidth=0, highlightthickness=0, command=start_live_detection)
start_button.image = start_btn_img
start_button.pack(pady=5)

# Button(root, text="Start Live Detection", command=start_live_detection).pack(pady=5)

# button to stop live detection, calls the stop_live_detection function when pressed
stop_btn_img = tk.PhotoImage(file="new buttons/stop.png")
stop_button = tk.Button(root, image=stop_btn_img, borderwidth=0, highlightthickness=0, command=stop_live_detection)
stop_button.image = stop_btn_img
stop_button.pack(pady=5)
# Button(root, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)

# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    # result_label.config(text="") # label below the image

clear_btn_img = tk.PhotoImage(file="new buttons/cls.png")
clear_button = tk.Button(root, image=clear_btn_img, borderwidth=0, highlightthickness=0, command=clear_canvas)
clear_button.image = clear_btn_img
clear_button.pack(pady=5)

# Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) # the button itself, calls the clear_canvas function

# exit button to exit the application
exit_btn_img = tk.PhotoImage(file="new buttons/cancel.png")
exit_button = tk.Button(root, image=exit_btn_img, borderwidth=0, highlightthickness=0, command=root.destroy)
exit_button.image = exit_btn_img
exit_button.pack(pady=5)

# Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.pack()

is_canvas_fullscreen = False

# Update layout
root.update_idletasks()

# Canvas size and position
canvas_width = 450
canvas_height = 450
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 10  # 10px gap

# Save original geometry
original_canvas_geometry = {
    "x": canvas_x,
    "y": canvas_y,
    "width": canvas_width,
    "height": canvas_height
}

# Create canvas
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="gray")
canvas.place(relx=0.5, y=canvas_y, anchor="n")

def center_canvas_image(event=None):
    if hasattr(canvas, "image_item_id"):
        canvas.coords(
            canvas.image_item_id,
            canvas.winfo_width() // 2,
            canvas.winfo_height() // 2
        )

canvas.bind("<Configure>", center_canvas_image)

# Toggle function
def toggle_canvas_fullscreen():
    global is_canvas_fullscreen

    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        fullscreen_button.place(relx=0.99, rely=0.01, anchor="ne")
        fullscreen_button.config(text="Exit Full Screen")
    else:
        canvas.place_forget()  # Fully reset placement first

        # Center the canvas horizontally using relx
        canvas.place(
            relx=0.5,
            y=original_canvas_geometry["y"],
            anchor="n",
            width=original_canvas_geometry["width"],
            height=original_canvas_geometry["height"]
        )

        fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")
        fullscreen_button.config(text="Enable Full Screen")

    root.update_idletasks()

    # Re-center image on canvas after it is resized
    canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)


# Place fullscreen toggle button on root (not canvas)
fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")

# Bind Escape key to exit fullscreen
root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

root.bind("<f>", lambda event: toggle_canvas_fullscreen())

def toggle_fullscreen(event=None):
    root.attributes("-fullscreen", not root.attributes("-fullscreen"))
    
import platform
# Platform-specific fullscreen toggle
if platform.system() == "Darwin":  # macOS
    root.bind("<Command-f>", toggle_fullscreen)
else:  # Windows or Linux
    root.bind("<Control-f>", toggle_fullscreen)

# result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist

        # center the image on canvas
        canvas.update()  # make sure canvas has correct width/height
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        # center the detection image on canvas
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        # result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            # result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            # result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            # result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

TclError: couldn't open "gun icon.png": no such file or directory

finalising

In [5]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
# import darkdetect
from tkinter import ttk
# from playsound import playsound
import pygame

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection
record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
# dark_mode = False
# theme_preference = "System"  # "Light", "Dark", or "System" the default
# is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)



# Tooltip class for hovering mouse for info and help button

class ToolTip:
    def __init__(self, widget, text):
        self.widget = widget
        self.text = text
        self.tip_window = None
        self.label = None
        widget.bind("<Enter>", self.schedule)
        widget.bind("<Leave>", self.hide_tip)

    def schedule(self, event=None):
        self.widget.after_idle(self.show_tip)

    def show_tip(self, event=None):
        if self.tip_window or not self.text:
            return

        self.tip_window = tw = tk.Toplevel(self.widget)
        tw.wm_overrideredirect(True)
        tw.attributes("-topmost", True)

        self.label = tk.Label(
            tw,
            text=self.text,
            background="#ffffe0",
            relief=tk.SOLID,
            borderwidth=1,
            font=("Helvetica", 10, "normal"),
            wraplength=250,
            padx=6,
            pady=3
        )
        self.label.pack(ipadx=1)

        tw.update_idletasks()

        x = self.widget.winfo_rootx() + 10
        y = self.widget.winfo_rooty() + self.widget.winfo_height() + 10

        screen_width = self.widget.winfo_screenwidth()
        screen_height = self.widget.winfo_screenheight()
        width = tw.winfo_width()
        height = tw.winfo_height()

        if x + width > screen_width:
            x = screen_width - width - 10
        if y + height > screen_height:
            y = screen_height - height - 10

        tw.geometry(f"+{x}+{y}")
        tw.update_idletasks()  # Force redraw (macOS needs this)

    def hide_tip(self, event=None):
        if self.tip_window:
            self.tip_window.destroy()
            self.tip_window = None


# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="logo.png")
root.iconphoto(True, icon_image)

# Load background image (match window size or resize as needed)
bg_image = Image.open("BG.png")  # loading image
bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
bg_photo = ImageTk.PhotoImage(bg_image)

# Create label and place it
bg_label = tk.Label(root, image=bg_photo)
bg_label.image = bg_photo  # keep reference
bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # full window

root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("600x750") # size of the application window
root.attributes('-fullscreen', True)
# def get_system_theme():
    # return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # theme selection
    # theme_frame = ttk.Frame(settings_window)
    # theme_frame.pack(pady=10)

    # ttk.Label(theme_frame, text="Theme: ").pack(side=tk.LEFT)

    # theme_options = ["System", "Light", "Dark"]
    # theme_var = tk.StringVar()
    # theme_var.set(theme_preference if theme_preference in theme_options else "System")
    # theme_menu = ttk.OptionMenu(theme_frame, theme_var, theme_var.get(), *theme_options)
    # theme_menu.pack(side=tk.LEFT)

    # def setup_ttk_styles(bg, fg):
    #     style = ttk.Style(settings_window)
    #     style.theme_use('default')  
    #     style.configure('.', background=bg, foreground=fg)
    #     style.configure('TLabel', background=bg, foreground=fg)
    #     style.configure('TButton', background=bg, foreground=fg)
    #     style.configure('TFrame', background=bg)
    #     style.configure('TSpinbox', fieldbackground=bg, foreground=fg, background=bg)
    #     style.configure('TMenubutton', background=bg, foreground=fg)

    # def apply_theme():
    #     global dark_mode

        # determine theme from preference
        # if theme_preference == "System":
        #     dark_mode = darkdetect.isDark()
        # else:
        #     dark_mode = (theme_preference == "Dark")

        # bg = "#2e2e2e" if dark_mode else "#f0f0f0"
        # fg = "#ffffff" if dark_mode else "#000000"

        # root.configure(bg=bg)
        # settings_button.configure(bg=bg, fg=fg)

        # settings_window.configure(bg=bg)

        # setup_ttk_styles(bg, fg)

    def save_settings():
        global selected_fps, theme_preference, record_buffer_seconds
        try:
            selected_fps = int(fps_spinbox.get())
            record_buffer_seconds = int(buffer_spinbox.get())
            # theme_preference = theme_var.get()
            # apply_theme()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)

    # apply_theme()  # apply current theme immediately on opening

    
# settings_button = tk.Button(root, text="⚙️", font=("Arial", 12), command=open_settings, bd=0, pady=7)
# settings_button.place(relx=1.0, x=-10, y=10, anchor="ne")


# main title displaying inside the application
# Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    # only the file types specified are allowed
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 


# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    # result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="Live Detection Stopped") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime


def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio
    import threading
    # from playsound import playsound

    pygame.mixer.init()  # initialize pygame mixer

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))  # circular buffer to store previous frames
    video_frames = []  # frames to save for recording
    recording = False
    recording_end_time = 0
    last_alert_time = -float("inf")


    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    # Create a persistent canvas image object once
    canvas.update_idletasks()
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    canvas_center_x = canvas_width // 2
    canvas_center_y = canvas_height // 2
    dummy_img = ImageTk.PhotoImage(Image.new("RGB", (400, 400)))
    canvas.image_item_id = canvas.create_image(canvas_center_x, canvas_center_y, anchor=tk.CENTER, image=dummy_img)

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        weapon_keywords = ("pistol", "assault rifle", "revolver", "shotgun")

        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if any(keyword in label for keyword in weapon_keywords):
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert to RGB for tkinter

        for box in results.boxes:
            if box.conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())  # store latest frame in buffer

        if Gun:
            # here 10 second means alarm will sound after 10 seconds of previous alarm
            if time.time() - last_alert_time > 5:
                def play_alert():
                    try:
                        pygame.mixer.music.load("siren alert.mp3")
                        pygame.mixer.music.play()
                    except Exception as e:
                        print("[ERROR] Playing sound:", e)

                threading.Thread(target=play_alert, daemon=True).start()
                last_alert_time = time.time()

            if not recording:
                recording = True
                recording_end_time = time.time() + 5  # record next 5 seconds
                video_frames = list(frame_buffer)  # include previous frames
            else:
                recording_end_time = time.time() + 5  # extend recording window

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))  # save video
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                pygame.mixer.music.stop()  # stop the siren after recording ends
                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # timestamp

        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  # show fps

        # img = Image.fromarray(frame_rgb)

        # Dynamically get current canvas size
        canvas.update_idletasks()
        canvas_width = canvas.winfo_width()
        canvas_height = canvas.winfo_height()
        
        img = Image.fromarray(frame_rgb)
       # Resize to fill the entire canvas (may stretch image slightly)
        img_resized = img.resize((canvas_width, canvas_height), Image.LANCZOS)
        imgtk = ImageTk.PhotoImage(img_resized)

        # img = img.resize((current_width, current_height), Image.LANCZOS)
        # imgtk = ImageTk.PhotoImage(img)

        canvas.imgtk = imgtk  # keep a reference so image is not garbage collected
        canvas.itemconfig(canvas.image_item_id, image=imgtk)

        canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)

        root.update_idletasks()  # update UI
        root.update()

# frame to group buttons to the left side
left_frame = tk.Frame(root)
left_frame.pack(side=tk.LEFT, anchor="n", padx=10, pady=50)

# button to select an image or a video from the computer
select_btn_img = tk.PhotoImage(file="new buttons/select.png")
select_button = tk.Button(left_frame, image=select_btn_img, borderwidth=0, highlightthickness=0, command=select_file)
select_button.image = select_btn_img
select_button.pack(pady=5)

# Button(left_frame, text="Select Image/Video", command=select_file).pack(pady=5) 

# button to start live detection, calls the start_live_detection function when pressed
start_btn_img = tk.PhotoImage(file="new buttons/start.png")
start_button = tk.Button(left_frame, image=start_btn_img, borderwidth=0, highlightthickness=0, command=start_live_detection)
start_button.image = start_btn_img
start_button.pack(pady=5)

# Button(left_frame, text="Start Live Detection", command=start_live_detection).pack(pady=5)


# button to stop live detection, calls the stop_live_detection function when pressed
stop_btn_img = tk.PhotoImage(file="new buttons/stop.png")
stop_button = tk.Button(left_frame, image=stop_btn_img, borderwidth=0, highlightthickness=0, command=stop_live_detection)
stop_button.image = stop_btn_img
stop_button.pack(pady=5)

# Button(left_frame, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)
normal_mode_buttons = []
essential_buttons = []

stop_sound_btn_img = tk.PhotoImage(file="new buttons/Soundoff.png")
stop_sound_button = tk.Button(root, image=stop_sound_btn_img, borderwidth=0, highlightthickness=0, command=lambda: pygame.mixer.music.stop())
stop_sound_button.image = stop_sound_btn_img
stop_sound_button.place(relx=1.0, rely=0.5, anchor="e")
# essential_buttons.append(stop_sound_button)

# stop_sound = tk.Button(root, text="Stop Sound", command=lambda: pygame.mixer.music.stop())
# stop_sound.pack(side=tk.RIGHT, anchor="e", padx=1, pady=5)

root.bind("<m>", lambda event: pygame.mixer.music.stop())


# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    # result_label.config(text="") # label below the image


# Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) 


# Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.place(relx=0.55,y=240, anchor='n')

is_canvas_fullscreen = False

# Update layout
root.update_idletasks()

# Canvas size and position
canvas_width = 750
canvas_height = 550
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 10  # 10px gap

# Save original geometry
# original_canvas_geometry = {
#     "x": canvas_x,
#     "y": canvas_y,
#     "width": canvas_width,
#     "height": canvas_height
# }



# Create canvas and place it at the bottom center
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="gray")
# canvas.place(relx=0.5, y=1, anchor="s")
canvas.place(
    relx=0.55,
    rely=0.6,
    anchor="center",
    width=canvas_width,
    height=canvas_height
)


root.update_idletasks()
original_canvas_geometry = {
    "width": canvas.winfo_width(),
    "height": canvas.winfo_height()
}



# Function to center image on canvas
def center_canvas_image(event=None):
    if hasattr(canvas, "image_item_id"):
        canvas.coords(
            canvas.image_item_id,
            canvas.winfo_width() // 2,
            canvas.winfo_height() // 2
        )

canvas.bind("<Configure>", center_canvas_image)


exit_fs_img = tk.PhotoImage(file="new buttons/ExitFS.png")

# to turn canvas into fullscreen mode
def toggle_canvas_fullscreen():
    global is_canvas_fullscreen

    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        
        lower_right_frame.place_forget()
        lower_right_frame.pack_forget()
        # show only essential buttons
        for btn in normal_mode_buttons:
            btn.pack_forget()
        for btn in essential_buttons:
            btn.pack(pady=5)
        stop_sound_button.pack_forget()
        stop_sound_button.grid_forget()
        stop_sound_button.place(relx=1.0, rely=0.5, anchor="e")
        fullscreen_button.config(image=exit_fs_img)  # change image to exit fullscreen image
    else:
        canvas.place_forget()
        canvas.place(
            relx=0.55,
            rely=0.6,
            anchor="center",
            width=original_canvas_geometry["width"],
            height=original_canvas_geometry["height"]
        )
        fullscreen_button.config(image=fs_btn_img)  # original image
        for btn in essential_buttons + normal_mode_buttons:
            btn.pack(pady=5)
        lower_right_frame.place(relx=1.0, rely=0.9, anchor="se")
        stop_sound_button.place(relx=1.0, rely=0.5, anchor="e")
    root.update_idletasks()



# frame to group buttons to the upper right side
upper_right_frame = tk.Frame(root)
upper_right_frame.place(relx=1.0, rely=0.05, anchor="ne")

# exit button to exit the application
exit_btn_img = tk.PhotoImage(file="new buttons/cancel.png")
exit_button = tk.Button(upper_right_frame, image=exit_btn_img, borderwidth=0, highlightthickness=0, command=root.destroy)
exit_button.image = exit_btn_img
exit_button.pack(pady=5)
essential_buttons.append(exit_button)

# button to turn canvas into full screen mode
fs_btn_img = tk.PhotoImage(file="new buttons/FS.png")
fullscreen_button = tk.Button(upper_right_frame, image=fs_btn_img, borderwidth=0, highlightthickness=0, command=toggle_canvas_fullscreen)
fullscreen_button.image = fs_btn_img
fullscreen_button.pack(pady=5)
essential_buttons.append(fullscreen_button)

# fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
# fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")

# the clear canvas button, calls the clear_canvas function
clear_btn_img = tk.PhotoImage(file="new buttons/CLS.png")
clear_button = tk.Button(upper_right_frame, image=clear_btn_img, borderwidth=0, highlightthickness=0, command=clear_canvas)
clear_button.image = clear_btn_img
clear_button.pack(pady=5)
normal_mode_buttons.append(clear_button)

# frame to group buttons to the upper right side
lower_right_frame = tk.Frame(root)
lower_right_frame.place(relx=1.0, rely=0.9, anchor="se")

settings_btn_img = tk.PhotoImage(file="new buttons/settings.png")
settings_button = tk.Button(lower_right_frame, image=settings_btn_img, borderwidth=0, highlightthickness=0, command=open_settings)
settings_button.image = settings_btn_img
settings_button.pack(pady=5)
normal_mode_buttons.append(settings_button)

info_btn_img = tk.PhotoImage(file="new buttons/info.png")
info_button = tk.Button(lower_right_frame, image=info_btn_img, borderwidth=0, highlightthickness=0)
info_button.image = info_btn_img
info_button.pack(pady=5)
normal_mode_buttons.append(info_button)

help_btn_img = tk.PhotoImage(file="new buttons/help.png")
help_button = tk.Button(lower_right_frame, image=help_btn_img, borderwidth=0, highlightthickness=0)
help_button.image = help_btn_img
help_button.pack(pady=5)
normal_mode_buttons.append(help_button)



if info_button.winfo_exists():
    ToolTip(info_button, """The Smart Weapon Detection System automatically /n 
                    detects weapons and keeps you safe by sending an /n
                    alarm as soon as a weapon is detected""")

if help_button.winfo_exists():
    ToolTip(help_button, """For any kind of help or instruction please/n 
                    e-mail us at smartdetection@help.com""")




# binding escape key to exit fullscreen when entered with ctrl f or cmnd f
root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

def on_f_key(event=None):
    toggle_canvas_fullscreen()
    root.attributes('-fullscreen', True)
    
# binding f key to enter or exit full canvas mode
root.bind("<f>", on_f_key)

def toggle_fullscreen(event=None):
    root.attributes("-fullscreen", not root.attributes("-fullscreen"))
    
import platform
# platform-specific fullscreen toggle
if platform.system() == "Darwin":  # macOS
    root.bind("<Command-f>", toggle_fullscreen)
else:  # Windows or Linux
    root.bind("<Control-f>", toggle_fullscreen)

# result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist

        # center the image on canvas
        canvas.update()  # make sure canvas has correct width/height
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        # center the detection image on canvas
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        # result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            # result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            # result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            # result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

# root.winfo_height() - canvas_height // 2 - 10

Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.12/tkinter/__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/pl/m919vj0j1_z39v5fc_pyrt1w0000gn/T/ipykernel_4244/1595649844.py", line 450, in <lambda>
    stop_sound_button = tk.Button(root, image=stop_sound_btn_img, borderwidth=0, highlightthickness=0, command=lambda: pygame.mixer.music.stop())
                                                                                                                       ^^^^^^^^^^^^^^^^^^^^^^^^^
pygame.error: mixer not initialized


resizing bg

In [1]:
import tkinter as tk
from PIL import Image, ImageTk

def resize_background(event):
    global bg_image, resized_bg, bg_label
    new_width = event.width
    new_height = event.height
    resized_bg = image.resize((new_width, new_height), Image.LANCZOS)
    bg_image = ImageTk.PhotoImage(resized_bg)
    bg_label.config(image=bg_image)

root = tk.Tk()
root.title("Background Image Example")

image = Image.open("BG.png")  # Replace "your_image.jpg" with your image file path
bg_image = ImageTk.PhotoImage(image)

bg_label = tk.Label(root, image=bg_image)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)
bg_label.bind("<Configure>", resize_background)

root.mainloop()

In [2]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
# import darkdetect
from tkinter import ttk
# from playsound import playsound
import pygame
import numpy as np

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection
record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
alarm_enabled = True  # by default the alarm is enabled
camera_source = "Webcam"  # default value
ip_camera_url = ""        # optionally pre-fill or leave empty


# dark_mode = False
# theme_preference = "System"  # "Light", "Dark", or "System" the default
# is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)



# Tooltip class for hovering mouse for info and help button

class ToolTip:
    def __init__(self, widget, text):
        self.widget = widget
        self.text = text
        self.tip_window = None
        self.label = None
        widget.bind("<Enter>", self.schedule)
        widget.bind("<Leave>", self.hide_tip)

    def schedule(self, event=None):
        self.widget.after_idle(self.show_tip)

    def show_tip(self, event=None):
        if self.tip_window or not self.text:
            return

        self.tip_window = tw = tk.Toplevel(self.widget)
        tw.wm_overrideredirect(True)
        tw.attributes("-topmost", True)

        self.label = tk.Label(
            tw,
            text=self.text,
            background="#ffffe0",
            relief=tk.SOLID,
            borderwidth=1,
            font=("Helvetica", 10, "normal"),
            wraplength=250,
            padx=6,
            pady=3
        )
        self.label.pack(ipadx=1)

        tw.update_idletasks()

        x = self.widget.winfo_rootx() + 10
        y = self.widget.winfo_rooty() + self.widget.winfo_height() + 10

        screen_width = self.widget.winfo_screenwidth()
        screen_height = self.widget.winfo_screenheight()
        width = tw.winfo_width()
        height = tw.winfo_height()

        if x + width > screen_width:
            x = screen_width - width - 10
        if y + height > screen_height:
            y = screen_height - height - 10

        tw.geometry(f"+{x}+{y}")
        tw.update_idletasks()  # Force redraw (macOS needs this)

    def hide_tip(self, event=None):
        if self.tip_window:
            self.tip_window.destroy()
            self.tip_window = None


# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="logo.png")
root.iconphoto(True, icon_image)

def resize_background(event):
    global bg_image, resized_bg, bg_label
    new_width = event.width
    new_height = event.height
    resized_bg = image.resize((new_width, new_height), Image.LANCZOS)
    bg_image = ImageTk.PhotoImage(resized_bg)
    bg_label.config(image=bg_image)

    
# Load background image (match window size or resize as needed)
image = Image.open("BG.png")  # loading image
# bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
bg_image = ImageTk.PhotoImage(image)




# # Create label and place it
# bg_label = tk.Label(root, image=bg_photo)
# bg_label.image = bg_photo  # keep reference
# bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # full window

bg_label = tk.Label(root, image=bg_image)
bg_label.image = bg_image  # keep reference
bg_label.place(x=0, y=0, relwidth=1, relheight=1)
bg_label.bind("<Configure>", resize_background)




root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("1200x850") # size of the application window
root.attributes('-fullscreen', True)
# def get_system_theme():
    # return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # check box to toggle alarm on or off
    alarm_var = tk.BooleanVar()
    alarm_var.set(alarm_enabled)
    
    alarm_check = ttk.Checkbutton(
        settings_window,
        text="Enable Alarm Sound",
        variable=alarm_var
    )
    alarm_check.pack(pady=10)

    # # Create a custom style
    # style = ttk.Style()
    # style.theme_use('clam')
    
    # # Customize the TMenubutton part of OptionMenu
    # style.configure('Custom.TMenubutton',
    #                 foreground='blue',  # text color
    #                 background='gray',  # background of button
    #                 font=('Arial', 12))

    
    # selecting camera source option
    camera_frame = tk.Frame(settings_window)
    camera_frame.pack(pady=10)
    
    tk.Label(camera_frame, text="Camera Source:").pack(side=tk.LEFT)
    
    camera_var = tk.StringVar(value=camera_source)
    camera_menu = ttk.OptionMenu(camera_frame, camera_var, camera_var.get(), "Webcam", "IP Camera")
    # camera_menu.configure(style='Custom.TMenubutton')
    camera_menu.pack(side=tk.LEFT)
    
    # IP camera URL input (shown only if "IP Camera" is selected)
    ip_url_var = tk.StringVar(value=ip_camera_url)
    ip_url_frame = ttk.Frame(settings_window)
    ip_url_entry = ttk.Entry(ip_url_frame, textvariable=ip_url_var, width=25)

    def toggle_ip_url_field(*args):
        if camera_var.get() == "IP Camera":
            ip_url_frame.pack(pady=5)
            ttk.Label(ip_url_frame, text="IP Camera URL:").pack(side=tk.LEFT)
            ip_url_entry.pack(side=tk.LEFT)
        else:
            ip_url_frame.pack_forget()
    
    camera_var.trace_add("write", toggle_ip_url_field)
    toggle_ip_url_field()  # call initially to hide/show



    def save_settings():
        global selected_fps, record_buffer_seconds, alarm_enabled, camera_source, ip_camera_url
        try:
            selected_fps = int(fps_spinbox.get()) # saving the fps set by user
            record_buffer_seconds = int(buffer_spinbox.get()) # saving the recoring seconds set by user
            alarm_enabled = alarm_var.get() # saving alarm preference
            camera_source = camera_var.get()
            ip_camera_url = ip_url_var.get()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s\nAlarm: {'On' if alarm_enabled else 'Off'}")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)



# main title displaying inside the application
# Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    # only the file types specified are allowed
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 


# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    # result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="Live Detection Stopped") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime


def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio
    import threading

    pygame.mixer.init()  # initialize pygame mixer

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))  # circular buffer to store previous frames
    video_frames = []  # frames to save for recording
    recording = False
    recording_end_time = 0
    last_alert_time = -float("inf")

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    # Remove canvas borders (still good to keep)
    canvas.config(bd=0, highlightthickness=0)

    # Create a persistent canvas image object once
    canvas.update_idletasks()
    dummy_img = ImageTk.PhotoImage(Image.new("RGB", (400, 400)))
    canvas.image_item_id = canvas.create_image(canvas.winfo_width() // 2, canvas.winfo_height() // 2, anchor=tk.CENTER, image=dummy_img)

    # Determine camera source
    if camera_source == "Webcam":
        cap = cv2.VideoCapture(0)
    else:
        cap = cv2.VideoCapture(ip_camera_url)

    if not cap.isOpened():
        print("[ERROR] Unable to open video stream")
        return

    def resize_with_padding(img, target_width, target_height, color=(0, 0, 0)):
        original_height, original_width = img.shape[:2]
        scale = min(target_width / original_width, target_height / original_height)
        new_width = int(original_width * scale)
        new_height = int(original_height * scale)
        resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
        result = np.full((target_height, target_width, 3), color, dtype=np.uint8)
        y_offset = (target_height - new_height) // 2
        x_offset = (target_width - new_width) // 2
        result[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_img
        return result

    while cap and live_running:
        # Dynamically get canvas size each time, with fallback
        canvas.update_idletasks()
        canvas_width = max(canvas.winfo_width(), 100)
        canvas_height = max(canvas.winfo_height(), 100)

        # Debug: print canvas size every frame
        print(f"Canvas size: {canvas_width} x {canvas_height}")

        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)[0]

        Gun = False
        weapon_keywords = ("pistol", "assault rifle", "revolver", "shotgun")

        for box in results.boxes:
            label = model.names[int(box.cls[0])].lower()
            if any(keyword in label for keyword in weapon_keywords):
                Gun = True
                break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert to RGB for tkinter

        for box in results.boxes:
            if box.conf > 0.55:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_rgb, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())  # store latest frame in buffer

        if Gun:
            # here 5 second means alarm will sound after 5 seconds of previous alarm
            if alarm_enabled and time.time() - last_alert_time > 5:
                def play_alert():
                    try:
                        pygame.mixer.music.load("siren alert.mp3")
                        pygame.mixer.music.play()
                    except Exception as e:
                        print("[ERROR] Playing sound:", e)

                threading.Thread(target=play_alert, daemon=True).start()
                last_alert_time = time.time()

            if not recording:
                recording = True
                recording_end_time = time.time() + 5  # record next 5 seconds
                video_frames = list(frame_buffer)  # include previous frames
            else:
                recording_end_time = time.time() + 5  # extend recording window

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))  # save video
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                pygame.mixer.music.stop()  # stop the siren after recording ends
                video_frames = []

        # Add overlays before resizing
        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        padded_frame = resize_with_padding(frame_rgb, canvas_width, canvas_height)
        imgtk = ImageTk.PhotoImage(Image.fromarray(padded_frame))

        canvas.imgtk = imgtk
        canvas.itemconfig(canvas.image_item_id, image=imgtk)
        canvas.coords(canvas.image_item_id, canvas_width // 2, canvas_height // 2)

        root.update_idletasks()
        root.update()







# frame to group buttons to the left side
left_frame = tk.Frame(root)
left_frame.pack(side=tk.LEFT, anchor="n", padx=10, pady=50)

# button to select an image or a video from the computer
select_btn_img = tk.PhotoImage(file="new buttons/Select.png")
select_button = tk.Button(left_frame, image=select_btn_img, borderwidth=0, highlightthickness=0, command=select_file)
select_button.image = select_btn_img
select_button.pack(pady=5)

# Button(left_frame, text="Select Image/Video", command=select_file).pack(pady=5) 

# button to start live detection, calls the start_live_detection function when pressed
start_btn_img = tk.PhotoImage(file="new buttons/start.png")
start_button = tk.Button(left_frame, image=start_btn_img, borderwidth=0, highlightthickness=0, command=start_live_detection)
start_button.image = start_btn_img
start_button.pack(pady=5)

# Button(left_frame, text="Start Live Detection", command=start_live_detection).pack(pady=5)


# button to stop live detection, calls the stop_live_detection function when pressed
stop_btn_img = tk.PhotoImage(file="new buttons/stop.png")
stop_button = tk.Button(left_frame, image=stop_btn_img, borderwidth=0, highlightthickness=0, command=stop_live_detection)
stop_button.image = stop_btn_img
stop_button.pack(pady=5)

# Button(left_frame, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)
normal_mode_buttons = []
essential_buttons = []

stop_sound_btn_img = tk.PhotoImage(file="new buttons/Soundoff.png")
stop_sound_button = tk.Button(root, image=stop_sound_btn_img, borderwidth=0, highlightthickness=0, command=lambda: pygame.mixer.music.stop())
stop_sound_button.image = stop_sound_btn_img
stop_sound_button.place(relx=1.0, rely=0.5, anchor="e")
# essential_buttons.append(stop_sound_button)

# stop_sound = tk.Button(root, text="Stop Sound", command=lambda: pygame.mixer.music.stop())
# stop_sound.pack(side=tk.RIGHT, anchor="e", padx=1, pady=5)

root.bind("<m>", lambda event: pygame.mixer.music.stop())


# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    # result_label.config(text="") # label below the image


# Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) 


# Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.place(relx=0.55,y=240, anchor='n')

is_canvas_fullscreen = False

# Update layout
root.update_idletasks()

# Canvas size and position
canvas_width = 750
canvas_height = 550
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 10  # 10px gap

# Save original geometry
# original_canvas_geometry = {
#     "x": canvas_x,
#     "y": canvas_y,
#     "width": canvas_width,
#     "height": canvas_height
# }



# Create canvas and place it at the bottom center
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="gray")

# canvas.place(
#     relx=0.55,
#     rely=0.6,
#     anchor="center",
#     width=canvas_width,
#     height=canvas_height
# )

# Create container frame at the exact position you want
container = tk.Frame(root, width=750, height=550)
container.place(relx=0.55, rely=0.6, anchor="center")

# Create canvas inside container, pack to fill it exactly
canvas = tk.Canvas(container, width=750, height=550)
canvas.pack(fill=tk.BOTH, expand=True)



root.update_idletasks()
original_canvas_geometry = {
    "width": canvas.winfo_width(),
    "height": canvas.winfo_height()
}



# Function to center image on canvas
def center_canvas_image(event=None):
    if hasattr(canvas, "image_item_id"):
        canvas.coords(
            canvas.image_item_id,
            canvas.winfo_width() // 2,
            canvas.winfo_height() // 2
        )

canvas.bind("<Configure>", center_canvas_image)


exit_fs_img = tk.PhotoImage(file="new buttons/ExitFS.png")

# to turn canvas into fullscreen mode
def toggle_canvas_fullscreen():
    global is_canvas_fullscreen

    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        
        lower_right_frame.place_forget()
        lower_right_frame.pack_forget()
        # show only essential buttons
        for btn in normal_mode_buttons:
            btn.pack_forget()
        for btn in essential_buttons:
            btn.pack(pady=5)
        stop_sound_button.pack_forget()
        stop_sound_button.grid_forget()
        stop_sound_button.place(relx=1.0, rely=0.5, anchor="e")
        stop_sound_button.lift()
        fullscreen_button.config(image=exit_fs_img)  # change image to exit fullscreen image
    else:
        canvas.place_forget()
        canvas.place(
            relx=0.55,
            rely=0.6,
            anchor="center",
            width=original_canvas_geometry["width"],
            height=original_canvas_geometry["height"]
        )
        fullscreen_button.config(image=fs_btn_img)  # original image
        for btn in essential_buttons + normal_mode_buttons:
            btn.pack(pady=5)
        lower_right_frame.place(relx=1.0, rely=0.9, anchor="se")
        stop_sound_button.place(relx=1.0, rely=0.5, anchor="e")
    root.update_idletasks()



# frame to group buttons to the upper right side
upper_right_frame = tk.Frame(root)
upper_right_frame.place(relx=1.0, rely=0.05, anchor="ne")

# exit button to exit the application
exit_btn_img = tk.PhotoImage(file="new buttons/cancel.png")
exit_button = tk.Button(upper_right_frame, image=exit_btn_img, borderwidth=0, highlightthickness=0, command=root.destroy)
exit_button.image = exit_btn_img
exit_button.pack(pady=5)
essential_buttons.append(exit_button)

# button to turn canvas into full screen mode
fs_btn_img = tk.PhotoImage(file="new buttons/FS.png")
fullscreen_button = tk.Button(upper_right_frame, image=fs_btn_img, borderwidth=0, highlightthickness=0, command=toggle_canvas_fullscreen)
fullscreen_button.image = fs_btn_img
fullscreen_button.pack(pady=5)
essential_buttons.append(fullscreen_button)

# fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
# fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")

# the clear canvas button, calls the clear_canvas function
clear_btn_img = tk.PhotoImage(file="new buttons/CLS.png")
clear_button = tk.Button(upper_right_frame, image=clear_btn_img, borderwidth=0, highlightthickness=0, command=clear_canvas)
clear_button.image = clear_btn_img
clear_button.pack(pady=5)
normal_mode_buttons.append(clear_button)

# frame to group buttons to the upper right side
lower_right_frame = tk.Frame(root)
lower_right_frame.place(relx=1.0, rely=0.9, anchor="se")

settings_btn_img = tk.PhotoImage(file="new buttons/settings.png")
settings_button = tk.Button(lower_right_frame, image=settings_btn_img, borderwidth=0, highlightthickness=0, command=open_settings)
settings_button.image = settings_btn_img
settings_button.pack(pady=5)
normal_mode_buttons.append(settings_button)

info_btn_img = tk.PhotoImage(file="new buttons/info.png")
info_button = tk.Button(lower_right_frame, image=info_btn_img, borderwidth=0, highlightthickness=0)
info_button.image = info_btn_img
info_button.pack(pady=5)
normal_mode_buttons.append(info_button)

help_btn_img = tk.PhotoImage(file="new buttons/help.png")
help_button = tk.Button(lower_right_frame, image=help_btn_img, borderwidth=0, highlightthickness=0)
help_button.image = help_btn_img
help_button.pack(pady=5)
normal_mode_buttons.append(help_button)



if info_button.winfo_exists():
    ToolTip(info_button, """The Smart Weapon Detection System automatically /n 
                    detects weapons and keeps you safe by sending an /n
                    alarm as soon as a weapon is detected""")

if help_button.winfo_exists():
    ToolTip(help_button, """For any kind of help or instruction please/n 
                    e-mail us at smartdetection@help.com""")


# binding escape key to exit fullscreen when entered with ctrl f or cmnd f
root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

def on_f_key(event=None):
    toggle_canvas_fullscreen()
    root.attributes('-fullscreen', True)
    
# binding f key to enter or exit full canvas mode
root.bind("<f>", on_f_key)

def toggle_fullscreen(event=None):
    root.attributes("-fullscreen", not root.attributes("-fullscreen"))
    
import platform
# platform-specific fullscreen toggle
if platform.system() == "Darwin":  # macOS
    root.bind("<Command-f>", toggle_fullscreen)
else:  # Windows or Linux
    root.bind("<Control-f>", toggle_fullscreen)

# result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist

        # center the image on canvas
        canvas.update()  # make sure canvas has correct width/height
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        # center the detection image on canvas
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        # result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            # result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            # result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            # result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()


Live detection started.
Canvas size: 750 x 550

0: 384x640 (no detections), 149.7ms
Speed: 5.8ms preprocess, 149.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Canvas size: 750 x 550

0: 384x640 (no detections), 149.9ms
Speed: 3.7ms preprocess, 149.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Canvas size: 750 x 550

0: 384x640 (no detections), 152.1ms
Speed: 2.7ms preprocess, 152.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Canvas size: 750 x 550

0: 384x640 (no detections), 154.7ms
Speed: 2.9ms preprocess, 154.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Canvas size: 750 x 550

0: 384x640 (no detections), 150.3ms
Speed: 3.0ms preprocess, 150.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Canvas size: 750 x 550

0: 384x640 (no detections), 151.8ms
Speed: 3.4ms preprocess, 151.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Canvas size: 750 x 550

0: 3

Exception in thread Thread-9 (show_live_frame):
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/opt/miniconda3/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/miniconda3/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/pl/m919vj0j1_z39v5fc_pyrt1w0000gn/T/ipykernel_5975/1539539593.py", line 365, in show_live_frame
  File "/opt/miniconda3/lib/python3.12/tkinter/__init__.py", line 1357, in winfo_width
    self.tk.call('winfo', 'width', self._w))
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: main thread is not in main loop


In [1]:
"""
Created on Thu Mar 27 00:20:35 2025

@author: Saad Amir
"""
import sys
import os
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading
from tkinter import messagebox
# import darkdetect
from tkinter import ttk
# from playsound import playsound
import pygame

cap = None # this will be used for storing the camera access
live_running = False # this for storing information about live detection
record_buffer_seconds = 5
recording = False
recording_writer = None
recording_start_time = None
recording_frames = []
selected_fps = 25 # setting default fps to 25 for live detection
alarm_enabled = True  # by default the alarm is enabled
camera_source = "Webcam"  # default value
ip_camera_url = ""        # optionally pre-fill or leave empty


# dark_mode = False
# theme_preference = "System"  # "Light", "Dark", or "System" the default
# is_dark_mode = darkdetect.isDark()  # Initial theme flag

# function to read the model to avoid errors in .exe files later
# we can use the normal way of reading model like model(yolo) but when we run the code
# as a stand alone we get error of no model found so this is the 
# best way to integrate model within the gui
def resource_path(relative_path):
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

# loading model via function
model_path = resource_path("best4.pt")
model = YOLO(model_path)



# Tooltip class for hovering mouse for info and help button

class ToolTip:
    def __init__(self, widget, text):
        self.widget = widget
        self.text = text
        self.tip_window = None
        self.label = None
        widget.bind("<Enter>", self.schedule)
        widget.bind("<Leave>", self.hide_tip)

    def schedule(self, event=None):
        self.widget.after_idle(self.show_tip)

    def show_tip(self, event=None):
        if self.tip_window or not self.text:
            return

        self.tip_window = tw = tk.Toplevel(self.widget)
        tw.wm_overrideredirect(True)
        tw.attributes("-topmost", True)

        self.label = tk.Label(
            tw,
            text=self.text,
            background="#ffffe0",
            relief=tk.SOLID,
            borderwidth=1,
            font=("Helvetica", 10, "normal"),
            wraplength=250,
            padx=6,
            pady=3
        )
        self.label.pack(ipadx=1)

        tw.update_idletasks()

        x = self.widget.winfo_rootx() + 10
        y = self.widget.winfo_rooty() + self.widget.winfo_height() + 10

        screen_width = self.widget.winfo_screenwidth()
        screen_height = self.widget.winfo_screenheight()
        width = tw.winfo_width()
        height = tw.winfo_height()

        if x + width > screen_width:
            x = screen_width - width - 10
        if y + height > screen_height:
            y = screen_height - height - 10

        tw.geometry(f"+{x}+{y}")
        tw.update_idletasks()  # Force redraw (macOS needs this)

    def hide_tip(self, event=None):
        if self.tip_window:
            self.tip_window.destroy()
            self.tip_window = None


# GUI setup
root = tk.Tk() # creating the UI window

icon_image = tk.PhotoImage(file="logo.png")
root.iconphoto(True, icon_image)

def resize_background(event):
    global bg_image, resized_bg, bg_label
    new_width = event.width
    new_height = event.height
    resized_bg = image.resize((new_width, new_height), Image.LANCZOS)
    bg_image = ImageTk.PhotoImage(resized_bg)
    bg_label.config(image=bg_image)

    
# Load background image (match window size or resize as needed)
image = Image.open("BG.png")  # loading image
# bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
bg_image = ImageTk.PhotoImage(image)




# # Create label and place it
# bg_label = tk.Label(root, image=bg_photo)
# bg_label.image = bg_photo  # keep reference
# bg_label.place(x=0, y=0, relwidth=1, relheight=1)  # full window

bg_label = tk.Label(root, image=bg_image)
bg_label.image = bg_image  # keep reference
bg_label.place(x=0, y=0, relwidth=1, relheight=1)
bg_label.bind("<Configure>", resize_background)




root.title("Smart Behavior and Weapon Detection") # title displaying on top of the window
root.geometry("1200x850") # size of the application window
root.attributes('-fullscreen', True)
# def get_system_theme():
    # return "Dark" if darkdetect.isDark() else "Light"

# defining a function to change settings like fps, recording time etc..
def open_settings():
    global selected_fps # accessing global variable

    settings_window = tk.Toplevel(root) # creating the settings window
    settings_window.title("Settings") # title on top of the window
    settings_window.geometry("300x300") # size of the settings window

    ttk.Label(settings_window, text="Settings Panel", font=("Arial", 14)).pack(pady=10) # label inside the settings window itself

    # FPS option
    fps_frame = ttk.Frame(settings_window)
    fps_frame.pack(pady=10)

    ttk.Label(fps_frame, text="Live Detection FPS: ").pack(side=tk.LEFT)

    fps_spinbox = ttk.Spinbox(fps_frame, from_=10, to=90, width=5)
    fps_spinbox.pack(side=tk.LEFT)
    fps_spinbox.delete(0, "end")
    fps_spinbox.insert(0, selected_fps)

    # buffer time option
    buffer_frame = ttk.Frame(settings_window)
    buffer_frame.pack(pady=10)

    ttk.Label(buffer_frame, text="Record Seconds (Pre & Post): ").pack(side=tk.LEFT)

    buffer_spinbox = ttk.Spinbox(buffer_frame, from_=1, to=60, width=5)
    buffer_spinbox.pack(side=tk.LEFT)
    buffer_spinbox.delete(0, "end")
    buffer_spinbox.insert(0, record_buffer_seconds)

    # check box to toggle alarm on or off
    alarm_var = tk.BooleanVar()
    alarm_var.set(alarm_enabled)
    
    alarm_check = ttk.Checkbutton(
        settings_window,
        text="Enable Alarm Sound",
        variable=alarm_var
    )
    alarm_check.pack(pady=10)

    # # Create a custom style
    # style = ttk.Style()
    # style.theme_use('clam')
    
    # # Customize the TMenubutton part of OptionMenu
    # style.configure('Custom.TMenubutton',
    #                 foreground='blue',  # text color
    #                 background='gray',  # background of button
    #                 font=('Arial', 12))

    
    # selecting camera source option
    camera_frame = tk.Frame(settings_window)
    camera_frame.pack(pady=10)
    
    tk.Label(camera_frame, text="Camera Source:").pack(side=tk.LEFT)
    
    camera_var = tk.StringVar(value=camera_source)
    camera_menu = ttk.OptionMenu(camera_frame, camera_var, camera_var.get(), "Webcam", "IP Camera")
    # camera_menu.configure(style='Custom.TMenubutton')
    camera_menu.pack(side=tk.LEFT)
    
    # IP camera URL input (shown only if "IP Camera" is selected)
    ip_url_var = tk.StringVar(value=ip_camera_url)
    ip_url_frame = ttk.Frame(settings_window)
    ip_url_entry = ttk.Entry(ip_url_frame, textvariable=ip_url_var, width=25)

    def toggle_ip_url_field(*args):
        if camera_var.get() == "IP Camera":
            ip_url_frame.pack(pady=5)
            ttk.Label(ip_url_frame, text="IP Camera URL:").pack(side=tk.LEFT)
            ip_url_entry.pack(side=tk.LEFT)
        else:
            ip_url_frame.pack_forget()
    
    camera_var.trace_add("write", toggle_ip_url_field)
    toggle_ip_url_field()  # call initially to hide/show



    def save_settings():
        global selected_fps, record_buffer_seconds, alarm_enabled, camera_source, ip_camera_url
        try:
            selected_fps = int(fps_spinbox.get()) # saving the fps set by user
            record_buffer_seconds = int(buffer_spinbox.get()) # saving the recoring seconds set by user
            alarm_enabled = alarm_var.get() # saving alarm preference
            camera_source = camera_var.get()
            ip_camera_url = ip_url_var.get()
            messagebox.showinfo("Settings Saved", f"FPS: {selected_fps}\nBuffer: {record_buffer_seconds}s\nAlarm: {'On' if alarm_enabled else 'Off'}")
            settings_window.destroy()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid FPS/Buffer value.")

    bf = ttk.Frame(settings_window)
    bf.pack(pady=10)
    tk.Button(bf, text="Save", command=save_settings).pack(side="left", padx=5)
    tk.Button(bf, text="Close", command=settings_window.destroy).pack(side="left", padx=5)



# main title displaying inside the application
# Label(root, text="Smart Behavior and Weapon Detection System", font=("Arial", 16)).pack(pady=10)

# creating a function to browse in the computer to select the file to perform detection upon (file selection)
def select_file():
    # only the file types specified are allowed
    file_path = filedialog.askopenfilename(filetypes=[('Images', '*.jpg *.png *.jpeg'), ('Videos', '*.mp4 *.avi *.mkv')])
    if file_path: # if file path is valid
        file_label.config(text=f"Selected: {file_path}") # just above the picture it displays path of the file selected or shows no file selected
        process_file(file_path) # calling the process file function on the selected picture to perform detection 


# function to start live deteciton
def start_live_detection():
    global cap, live_running # declaring variables as global variables
    cap = cv2.VideoCapture(0) # this opens the default camera
    if not cap.isOpened(): # if there is an error opening the default camera
        # result_label.config(text="Camera not accessible") # a label below the picture will say the text specified
        return
    live_running = True # set live running to true while live detection is on
    threading.Thread(target=show_live_frame, daemon=True).start() # running the live detection on a seperate thread of cpu for better performance 
    file_label.config(text="Doing Live Detection") # display the text specified above the picture

# function to stop detection
def stop_live_detection():
    global cap, live_running, recording, recording_writer, recording_frames # accessing global variables
    live_running = False # setting live running to false because now we are stopping live detection
    if cap: # checking if there is something in th ecap variable
        cap.release() # releasing all frames
    canvas.delete("all") # clearing the canvas (the gray frame in the UI)
    # result_label.config(text="Live detection stopped.") # text at the bottom of the application
    file_label.config(text="Live Detection Stopped") # reset the file label if previously any file was selected

    # if recording is in progress, save it now
    if recording and recording_writer:
        for f in recording_frames:
            recording_writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        recording_writer.release()
        print("Recording saved on stop.")
    recording = False # setting the recording variable back to false
    recording_writer = None
    recording_frames = []

prev_time = None
# display frames of camera and run model upon also save 5 seconds pre and post weapon detection
from datetime import datetime


def show_live_frame():
    global cap, live_running, selected_fps, record_buffer_seconds

    from collections import deque
    import time
    import imageio
    import threading
    # from playsound import playsound

    pygame.mixer.init()  # initialize pygame mixer

    fps = selected_fps
    buffer_seconds = record_buffer_seconds

    frame_buffer = deque(maxlen=int(buffer_seconds * fps))  # circular buffer to store previous frames
    video_frames = []  # frames to save for recording
    recording = False
    recording_end_time = 0
    last_alert_time = -float("inf")

    if not os.path.exists("recordings"):
        os.makedirs("recordings")

    print("Live detection started.")

    prev_time = time.time()

    # Create a persistent canvas image object once
    canvas.update_idletasks()
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    canvas_center_x = canvas_width // 2
    canvas_center_y = canvas_height // 2
    dummy_img = ImageTk.PhotoImage(Image.new("RGB", (400, 400)))
    canvas.image_item_id = canvas.create_image(canvas_center_x, canvas_center_y, anchor=tk.CENTER, image=dummy_img)

    # Determine camera source
    if camera_source == "Webcam":
        cap = cv2.VideoCapture(0)
    else:
        cap = cv2.VideoCapture(ip_camera_url)

    if not cap.isOpened():
        print("[ERROR] Unable to open video stream")
        return

    while cap and live_running:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        elapsed_time = current_time - prev_time
        if elapsed_time < 1.0 / fps:
            time.sleep(1.0 / fps - elapsed_time)
        prev_time = current_time

        results = model(frame)
        result = results[0]  # single-frame detection result

        Gun = False
        weapon_keywords = ("assault rifle", "revolver", 'pistol')
        allowed_classes = ['Pistol', 'Revolver', 'Assault Rifle']

        # Check if any detection boxes exist
        if result.boxes is not None and len(result.boxes) > 0:
            # Loop through detections
            for box in result.boxes:
                label = model.names[int(box.cls[0])].lower()

                # Check for weapon keywords
                if any(keyword in label for keyword in weapon_keywords):
                    Gun = True

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert to RGB for tkinter

        if result.boxes is not None and len(result.boxes) > 0:
            for box in result.boxes:
                class_id = int(box.cls[0])
                class_name = model.names[class_id]

                if box.conf[0] > 0.70 and class_name in allowed_classes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{class_name} {box.conf[0]:.2f}"
                    cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame_rgb, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_buffer.append(frame_rgb.copy())  # store latest frame in buffer

        if Gun:
            # here 5 second means alarm will sound after 5 seconds of previous alarm
            if alarm_enabled and time.time() - last_alert_time > 5:
                def play_alert():
                    try:
                        pygame.mixer.music.load("siren alert.mp3")
                        pygame.mixer.music.play()
                    except Exception as e:
                        print("[ERROR] Playing sound:", e)

                threading.Thread(target=play_alert, daemon=True).start()
                last_alert_time = time.time()

            if not recording:
                recording = True
                recording_end_time = time.time() + 5  # record next 5 seconds
                video_frames = list(frame_buffer)  # include previous frames
            else:
                recording_end_time = time.time() + 5  # extend recording window

        if recording:
            video_frames.append(frame_rgb.copy())

            if time.time() > recording_end_time:
                recording = False
                timestamp = int(time.time())
                out_path = os.path.join("recordings", f"detection_{timestamp}.mp4")

                try:
                    imageio.mimsave(out_path, video_frames, fps=int(fps))  # save video
                    print(f"[SAVED] Recording saved: {out_path}")
                except Exception as e:
                    print("[ERROR] Failed to save recording:", e)

                pygame.mixer.music.stop()  # stop the siren after recording ends
                video_frames = []

        cv2.putText(frame_rgb, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # timestamp

        fps_text = f"FPS: {fps:.2f}"
        text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_x = frame_rgb.shape[1] - text_size[0] - 10
        cv2.putText(frame_rgb, fps_text, (text_x, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)  # show fps

        # img = Image.fromarray(frame_rgb)

        # Dynamically get current canvas size
        canvas.update_idletasks()
        canvas_width = canvas.winfo_width()
        canvas_height = canvas.winfo_height()

        img = Image.fromarray(frame_rgb)
        # Resize to fill the entire canvas (may stretch image slightly)
        img_resized = img.resize((canvas_width, canvas_height), Image.LANCZOS)
        imgtk = ImageTk.PhotoImage(img_resized)

        # img = img.resize((current_width, current_height), Image.LANCZOS)
        # imgtk = ImageTk.PhotoImage(img)

        canvas.imgtk = imgtk  # keep a reference so image is not garbage collected
        canvas.itemconfig(canvas.image_item_id, image=imgtk)

        canvas.coords(canvas.image_item_id, canvas.winfo_width() // 2, canvas.winfo_height() // 2)

        root.update_idletasks()  # update UI
        root.update()


# frame to group buttons to the left side
left_frame = tk.Frame(root)
left_frame.pack(side=tk.LEFT, anchor="n", padx=10, pady=50)

# button to select an image or a video from the computer
select_btn_img = tk.PhotoImage(file="new buttons/Select.png")
select_button = tk.Button(left_frame, image=select_btn_img, borderwidth=0, highlightthickness=0, command=select_file)
select_button.image = select_btn_img
select_button.pack(pady=5)

# Button(left_frame, text="Select Image/Video", command=select_file).pack(pady=5) 

# button to start live detection, calls the start_live_detection function when pressed
start_btn_img = tk.PhotoImage(file="new buttons/start.png")
start_button = tk.Button(left_frame, image=start_btn_img, borderwidth=0, highlightthickness=0, command=start_live_detection)
start_button.image = start_btn_img
start_button.pack(pady=5)

# Button(left_frame, text="Start Live Detection", command=start_live_detection).pack(pady=5)


# button to stop live detection, calls the stop_live_detection function when pressed
stop_btn_img = tk.PhotoImage(file="new buttons/stop.png")
stop_button = tk.Button(left_frame, image=stop_btn_img, borderwidth=0, highlightthickness=0, command=stop_live_detection)
stop_button.image = stop_btn_img
stop_button.pack(pady=5)

# Button(left_frame, text="Stop Live Detection", command=stop_live_detection).pack(pady=5)
normal_mode_buttons = []
essential_buttons = []

stop_sound_btn_img = tk.PhotoImage(file="new buttons/Soundoff.png")
stop_sound_button = tk.Button(root, image=stop_sound_btn_img, borderwidth=0, highlightthickness=0, command=lambda: pygame.mixer.music.stop())
stop_sound_button.image = stop_sound_btn_img
stop_sound_button.place(relx=1.0, rely=0.5, anchor="e")
# essential_buttons.append(stop_sound_button)

# stop_sound = tk.Button(root, text="Stop Sound", command=lambda: pygame.mixer.music.stop())
# stop_sound.pack(side=tk.RIGHT, anchor="e", padx=1, pady=5)

root.bind("<m>", lambda event: pygame.mixer.music.stop())


# function that will clear the ui, like images that are stuck in the canvas or the file labels
def clear_canvas():
    canvas.delete("all") # command that deletes everything on the canvas
    canvas.image = None # setting image back to none
    file_label.config(text="No file selected") # resetting the file label on top of the canvas
    # result_label.config(text="") # label below the image


# Button(root, text="Clear Screen", command=clear_canvas).pack(pady=5) 


# Button(root, text="Exit", command=root.destroy).pack(pady=5)

# path or name of the file selected by the user shown above the canvas image,
# if none is selected it displays text specified
file_label = Label(root, text="No file selected", fg="red")
file_label.place(relx=0.55,y=240, anchor='n')

is_canvas_fullscreen = False

# Update layout
root.update_idletasks()

# Canvas size and position
canvas_width = 750
canvas_height = 550
canvas_x = (root.winfo_width() - canvas_width) // 2
canvas_y = file_label.winfo_y() + file_label.winfo_height() + 10  # 10px gap

# Save original geometry
# original_canvas_geometry = {
#     "x": canvas_x,
#     "y": canvas_y,
#     "width": canvas_width,
#     "height": canvas_height
# }



# Create canvas and place it at the bottom center
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="gray")
# canvas.place(relx=0.5, y=1, anchor="s")
canvas.place(
    relx=0.55,
    rely=0.6,
    anchor="center",
    width=canvas_width,
    height=canvas_height
)


root.update_idletasks()
original_canvas_geometry = {
    "width": canvas.winfo_width(),
    "height": canvas.winfo_height()
}



# Function to center image on canvas
def center_canvas_image(event=None):
    if hasattr(canvas, "image_item_id"):
        canvas.coords(
            canvas.image_item_id,
            canvas.winfo_width() // 2,
            canvas.winfo_height() // 2
        )

canvas.bind("<Configure>", center_canvas_image)


exit_fs_img = tk.PhotoImage(file="new buttons/ExitFS.png")

# to turn canvas into fullscreen mode
def toggle_canvas_fullscreen():
    global is_canvas_fullscreen

    is_canvas_fullscreen = not is_canvas_fullscreen

    if is_canvas_fullscreen:
        canvas.place(x=0, y=0, relwidth=1, relheight=1)
        
        lower_right_frame.place_forget()
        lower_right_frame.pack_forget()
        # show only essential buttons
        for btn in normal_mode_buttons:
            btn.pack_forget()
        for btn in essential_buttons:
            btn.pack(pady=5)
        stop_sound_button.pack_forget()
        stop_sound_button.grid_forget()
        stop_sound_button.place(relx=1.0, rely=0.5, anchor="e")
        stop_sound_button.lift()
        fullscreen_button.config(image=exit_fs_img)  # change image to exit fullscreen image
    else:
        canvas.place_forget()
        canvas.place(
            relx=0.55,
            rely=0.6,
            anchor="center",
            width=original_canvas_geometry["width"],
            height=original_canvas_geometry["height"]
        )
        fullscreen_button.config(image=fs_btn_img)  # original image
        for btn in essential_buttons + normal_mode_buttons:
            btn.pack(pady=5)
        lower_right_frame.place(relx=1.0, rely=0.9, anchor="se")
        stop_sound_button.place(relx=1.0, rely=0.5, anchor="e")
    root.update_idletasks()



# frame to group buttons to the upper right side
upper_right_frame = tk.Frame(root)
upper_right_frame.place(relx=1.0, rely=0.05, anchor="ne")

# exit button to exit the application
exit_btn_img = tk.PhotoImage(file="new buttons/cancel.png")
exit_button = tk.Button(upper_right_frame, image=exit_btn_img, borderwidth=0, highlightthickness=0, command=root.destroy)
exit_button.image = exit_btn_img
exit_button.pack(pady=5)
essential_buttons.append(exit_button)

# button to turn canvas into full screen mode
fs_btn_img = tk.PhotoImage(file="new buttons/FS.png")
fullscreen_button = tk.Button(upper_right_frame, image=fs_btn_img, borderwidth=0, highlightthickness=0, command=toggle_canvas_fullscreen)
fullscreen_button.image = fs_btn_img
fullscreen_button.pack(pady=5)
essential_buttons.append(fullscreen_button)

# fullscreen_button = tk.Button(root, text="Enable Full Screen", command=toggle_canvas_fullscreen)
# fullscreen_button.place(relx=0.99, rely=0.99, anchor="se")

# the clear canvas button, calls the clear_canvas function
clear_btn_img = tk.PhotoImage(file="new buttons/CLS.png")
clear_button = tk.Button(upper_right_frame, image=clear_btn_img, borderwidth=0, highlightthickness=0, command=clear_canvas)
clear_button.image = clear_btn_img
clear_button.pack(pady=5)
normal_mode_buttons.append(clear_button)

# frame to group buttons to the upper right side
lower_right_frame = tk.Frame(root)
lower_right_frame.place(relx=1.0, rely=0.9, anchor="se")

settings_btn_img = tk.PhotoImage(file="new buttons/settings.png")
settings_button = tk.Button(lower_right_frame, image=settings_btn_img, borderwidth=0, highlightthickness=0, command=open_settings)
settings_button.image = settings_btn_img
settings_button.pack(pady=5)
normal_mode_buttons.append(settings_button)

info_btn_img = tk.PhotoImage(file="new buttons/info.png")
info_button = tk.Button(lower_right_frame, image=info_btn_img, borderwidth=0, highlightthickness=0)
info_button.image = info_btn_img
info_button.pack(pady=5)
normal_mode_buttons.append(info_button)

help_btn_img = tk.PhotoImage(file="new buttons/help.png")
help_button = tk.Button(lower_right_frame, image=help_btn_img, borderwidth=0, highlightthickness=0)
help_button.image = help_btn_img
help_button.pack(pady=5)
normal_mode_buttons.append(help_button)



if info_button.winfo_exists():
    ToolTip(info_button, """The Smart Weapon Detection System automatically /n 
                    detects weapons and keeps you safe by sending an /n
                    alarm as soon as a weapon is detected""")

if help_button.winfo_exists():
    ToolTip(help_button, """For any kind of help or instruction please/n 
                    e-mail us at smartdetection@help.com""")


# binding escape key to exit fullscreen when entered with ctrl f or cmnd f
root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

def on_f_key(event=None):
    toggle_canvas_fullscreen()
    root.attributes('-fullscreen', True)
    
# binding f key to enter or exit full canvas mode
root.bind("<f>", on_f_key)

def toggle_fullscreen(event=None):
    root.attributes("-fullscreen", not root.attributes("-fullscreen"))
    
import platform
# platform-specific fullscreen toggle
if platform.system() == "Darwin":  # macOS
    root.bind("<Command-f>", toggle_fullscreen)
else:  # Windows or Linux
    root.bind("<Control-f>", toggle_fullscreen)

# result_label = Label(root, text="", fg="green")
# result_label.pack()

# Image/Video Detection Handler
def process_file(file_path):
    # display and open image preview
    if file_path.lower().endswith(('jpg', 'jpeg', 'png')):
        img = Image.open(file_path) # opening image using PILLOW
        img.thumbnail((400, 400)) # resizing image
        imgtk = ImageTk.PhotoImage(img) # Keeps the image saved so garbage collection dont remove it
        canvas.imgtk = imgtk  # store as an attribute of canvas to persist

        # center the image on canvas
        canvas.update()  # make sure canvas has correct width/height
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=canvas.imgtk) # display image on canvas

        # run the model and get results
        results = model(file_path, save=True, project="detections", name="results", exist_ok=True)
        boxes = results[0].boxes
        num_detections = len(boxes)
        
        # draw boxes on image
        frame = cv2.imread(file_path)
        for box in boxes:
            if box.conf >= 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # convert to RGB and display in canvas
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = img.resize((400, 400))
        imgtk = ImageTk.PhotoImage(img)

        # center the detection image on canvas
        cx = canvas.winfo_width() // 2
        cy = canvas.winfo_height() // 2
        canvas.create_image(cx, cy, anchor=tk.CENTER, image=imgtk)
        canvas.image = imgtk # Keeps the image saved so garbage collection dont remove it
        
        # update label at the bottom of the application
        # result_label.config(text="Detection Complete! Check 'detections/' folder.")
        
        # result_label.config(text="Detection Complete! Check 'detections/' folder.") # message at the bottom of screen
        # this command refreshes the ui before moving to the next code, before doing this the result
        # of the detection was not updating until i clicked ok
        root.update_idletasks()
        # popup message
        messagebox.showinfo("Detection Complete",
                            f"Weapons Detected: {num_detections}\n"
                            f"Results saved in: detections/results/")
    
    elif file_path.lower().endswith(('mp4', 'avi', 'mkv', 'gif')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            # result_label.config(text="Failed to open video.")
            return
    
        import datetime
        import imageio.v2 as imageio  # safer import
        video_frames = []
        total_detections = 0
    
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0 or fps > 240:
            fps = 20
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            results = model(frame)[0]
    
            for box in results.boxes:
                if box.conf >= 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    label = f"{model.names[int(box.cls[0])]} {box.conf[0]:.2f}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    total_detections += 1
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
    
        cap.release()
    
        os.makedirs("detections", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = os.path.join("detections", f"video_result_{timestamp}.mp4")
    
        try:
            imageio.mimsave(out_path, video_frames, fps=int(fps))
            # result_label.config(text=f"Video Detection Complete! Saved to {out_path}")
            messagebox.showinfo("Detection Complete",
                                f"Weapons Detected: {total_detections}\n"
                                f"Results saved in: {out_path}")
        except Exception as e:
            # result_label.config(text="Failed to save video.")
            messagebox.showerror("Save Error", f"Could not save video.\nError: {e}")


# Start GUI loop
root.mainloop()

# root.winfo_height() - canvas_height // 2 - 10

pygame 2.6.1 (SDL 2.28.4, Python 3.12.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
